# Apply SAM modes from ISODISTORT output file to a supercell structure compatible with VASP

In [2]:
import numpy as np

def matrix_to_string(matrix):
    # Converting each row of the matrix to a string
    row_strings = [" ".join(f"{value:.16f}" for value in row) for row in matrix]

    # Joining all row strings with a newline character
    return "\n".join(row_strings)

def string_to_matrix(string):
    string = [list(map(float, row.split())) for row in string.strip().split('\n')]

    # Converting the data to a NumPy matrix
    return np.array(string)

## Expand primitive to supercell


In [3]:
def cell_expand(primitive_lattice):
    transformation_matrix = np.array([
    [np.sqrt(2), 0, 0],
    [0, np.sqrt(2), 0],
    [0, 0, 2]
    ])
    #def expand_lattice_param(primitive_lattice, transformation_matrix):
    primitive_lattice = string_to_matrix(primitive_lattice)

    # Performing the matrix multiplication
    lattice_basis = np.dot(transformation_matrix, primitive_lattice)

    # Convert the matrix to a string in the desired format
    formatted_string = matrix_to_string(lattice_basis)

    print(formatted_string)
    return lattice_basis



In [4]:
def RXcell_expand(primitive_lattice):
    transformation_matrix = np.array([
    [2, 0, 0],
    [0, 2, 0],
    [0, 0, 2]
    ])
    #def expand_lattice_param(primitive_lattice, transformation_matrix):
    primitive_lattice = string_to_matrix(primitive_lattice)

    # Performing the matrix multiplication
    lattice_basis = np.dot(transformation_matrix, primitive_lattice)

    # Convert the matrix to a string in the desired format
    formatted_string = matrix_to_string(lattice_basis)

    print(formatted_string)
    return lattice_basis

In [32]:
NHNNO_prim_lattice = '''
     3.8392325820324888    0.0000000000000000    0.0000000000000000
     0.0000000000000000    3.8392325820324888    0.0000000000000000
     0.0000000000000000    0.0000000000000000   11.0776823720272581
'''
NHNNO_RX_lattice = RXcell_expand(NHNNO_prim_lattice)
NHNNO_super_lattice = cell_expand(NHNNO_prim_lattice)

7.6784651640649777 0.0000000000000000 0.0000000000000000
0.0000000000000000 7.6784651640649777 0.0000000000000000
0.0000000000000000 0.0000000000000000 22.1553647440545163


In [6]:
# Input data as a multiline string
NaNNO_prim_lattice = """
3.8280824710192780    0.0000000000000000    0.0000000000000000
0.0000000000000000    3.8280824710192780    0.0000000000000000
0.0000000000000000    0.0000000000000000   10.9441331724216901
"""
NaNNO_RX_lattice = RXcell_expand(NaNNO_prim_lattice)
NaNNO_super_lattice = cell_expand(NaNNO_prim_lattice)

7.6561649420385560 0.0000000000000000 0.0000000000000000
0.0000000000000000 7.6561649420385560 0.0000000000000000
0.0000000000000000 0.0000000000000000 21.8882663448433803
5.4137261483981742 0.0000000000000000 0.0000000000000000
0.0000000000000000 5.4137261483981742 0.0000000000000000
0.0000000000000000 0.0000000000000000 21.8882663448433803


In [7]:
NaNNO_prim_positions = '''
  0.5000000000000000  0.5000000000000000  0.0000000000000000
  0.5000000000000000  0.5000000000000000  0.5000000000000000
  0.0000000000000000  0.0000000000000000  0.2935347165073036
 -0.0000000000000000 -0.0000000000000000  0.7064652834926964
  0.0000000000000000 -0.0000000000000000  0.5000000000000000
  0.5000000000000000 -0.0000000000000000  0.6645394444672751
 -0.0000000000000000  0.5000000000000000  0.6645394444672751
  0.0000000000000000  0.5000000000000000  0.3354605255327224
  0.5000000000000000  0.0000000000000000  0.3354605255327224
  0.0000000000000000  0.0000000000000000  0.1299886821043846
 -0.0000000000000000 -0.0000000000000000  0.8700113178956155
'''

#### Expanasion for a tetragonal cell

In [8]:
# Transformation matrix t
transformation_matrix = np.array([
    [1, 1, 0],
    [1, -1, 0],
    [0, 0, 2]
])

def transformation(data_str):
    # Splitting the data into rows, separating labels from numerical data
    labels, numerical_data = [], []
    for row in data_str.strip().split('\n'):
        elements = row.split()
        labels.append(elements[0])  # Extract label
        numerical_data.append(list(map(float, elements[1:])))  # Convert remaining elements to floats

    # Converting the numerical data to a NumPy matrix
    matrix = np.array(numerical_data)

    # Calculating the inverse of the transformation matrix
    inverse_t = np.linalg.inv(transformation_matrix)

    # Performing the matrix multiplication
    result = np.dot(inverse_t, matrix.T).T

    # Transpose back and reattach the labels
    result_labeled = np.column_stack((labels, result))
    return result, result_labeled

def shifting(matrix):
    # Apply the first transformation (add 0.5 to the third column)
    transformed_matrix_z = np.copy(matrix)
    transformed_matrix_z[:, 2] += 0.5  # Adjusted index for mixed type

    # Intertwine the rows from the first transformation
    intertwined_matrix_z = np.vstack([np.vstack([row, transformed_row]) for row, transformed_row in zip(matrix, transformed_matrix_z)])

    # Apply the second transformation (add 0.5 to the first and second columns)
    def second_transformation(row):
        new_row = np.copy(row)
        new_row[0] += 0.5  # Adjusted index for mixed type
        new_row[1] += 0.5  # Adjusted index for mixed type
        return new_row

    intertwined_matrix_xy = np.vstack([np.vstack([row, second_transformation(row)]) for row in intertwined_matrix_z])
    intertwined_matrix_xy = intertwined_matrix_xy%1
    return intertwined_matrix_xy


def expand_positions(data_str):
    result, result_labeled = transformation(data_str)

    label_to_data = {}

    for row in result_labeled:
        elements = row
        label = elements[0]  # Extract label
        numerical_values = list(map(float, elements[1:]))  # Convert remaining elements to floats
        # Add numerical data to corresponding label list
        if label in label_to_data:
            label_to_data[label].append(numerical_values)
        else:
            label_to_data[label] = [numerical_values]

    # Print unique labels
    print("Unique Labels:", label_to_data.keys())


    # Apply the transformation and intertwine the rows
    unlabled_positions = np.vstack([shifting(label_to_data[lables]) for lables in label_to_data.keys()])


    # Apply the transformation and intertwine the rows
    transformed_arrays = []
    for label in label_to_data.keys():
        transformed_data = shifting(label_to_data[label])

        # Prepend each row with the label
        labeled_data = np.hstack([[[label]] * transformed_data.shape[0], transformed_data])  # Repeat label for each row
        transformed_arrays.append(labeled_data)

    # Stack the transformed arrays vertically
    labled_positions = np.vstack(transformed_arrays)

    # Convert the matrix to a string in the desired format
    formatted_num = matrix_to_string(unlabled_positions)
    print(formatted_num)
    return unlabled_positions, labled_positions

In [9]:

def RXtransformation(data_str):

    # Transformation matrix t
    transformation_matrix = np.array([
        [2, 0, 0],
        [0, 2, 0],
        [0, 0, 2]
    ])

    # Splitting the data into rows, separating labels from numerical data
    labels, numerical_data = [], []
    for row in data_str.strip().split('\n'):
        elements = row.split()
        labels.append(elements[0])  # Extract label
        numerical_data.append(list(map(float, elements[1:])))  # Convert remaining elements to floats

    # Converting the numerical data to a NumPy matrix
    matrix = np.array(numerical_data)

    # Calculating the inverse of the transformation matrix
    inverse_t = np.linalg.inv(transformation_matrix)

    # Performing the matrix multiplication
    result = np.dot(inverse_t, matrix.T).T

    # Transpose back and reattach the labels
    result_labeled = np.column_stack((labels, result))
    return result, result_labeled

def RXshifting(matrix):
    # Apply the first transformation (add 0.5 to the third column)
    transformed_matrix_z = np.copy(matrix)
    transformed_matrix_z[:, 2] += 0.5  # Adjusted index for mixed type

    # Intertwine the rows from the first transformation
    intertwined_matrix_z = np.vstack([np.vstack([row, transformed_row]) for row, transformed_row in zip(matrix, transformed_matrix_z)])

    # Apply the second transformation (add 0.5 to the first and second columns)
    def second_transformation(row):
        new_row = np.copy(row)
        #
        #new_row[0] += 0.5  # Adjusted index for mixed type
        new_row[1] += 0.5  # Adjusted index for mixed type
        return new_row
    
    def third_transformation(row):
        new_row = np.copy(row)
        #
        new_row[0] += 0.5  # Adjusted index for mixed type
        #new_row[1] += 0.5  # Adjusted index for mixed type
        return new_row

    intertwined_matrix_xy1 = np.vstack([np.vstack([row, second_transformation(row)]) for row in intertwined_matrix_z])
    intertwined_matrix_xy2 = np.vstack([np.vstack([row, third_transformation(row)]) for row in intertwined_matrix_xy1])
    intertwined_matrix_xy = intertwined_matrix_xy2%1
    return intertwined_matrix_xy


def RXexpand_positions(data_str):
    result, result_labeled = RXtransformation(data_str)

    label_to_data = {}

    for row in result_labeled:
        elements = row
        label = elements[0]  # Extract label
        numerical_values = list(map(float, elements[1:]))  # Convert remaining elements to floats
        # Add numerical data to corresponding label list
        if label in label_to_data:
            label_to_data[label].append(numerical_values)
        else:
            label_to_data[label] = [numerical_values]

    # Print unique labels
    print("Unique Labels:", label_to_data.keys())


    # Apply the transformation and intertwine the rows
    unlabled_positions = np.vstack([RXshifting(label_to_data[lables]) for lables in label_to_data.keys()])


    # Apply the transformation and intertwine the rows
    transformed_arrays = []
    for label in label_to_data.keys():
        transformed_data = RXshifting(label_to_data[label])

        # Prepend each row with the label
        labeled_data = np.hstack([[[label]] * transformed_data.shape[0], transformed_data])  # Repeat label for each row
        transformed_arrays.append(labeled_data)

    # Stack the transformed arrays vertically
    labled_positions = np.vstack(transformed_arrays)

    # Convert the matrix to a string in the desired format
    formatted_num = matrix_to_string(unlabled_positions)
    print(formatted_num)
    return unlabled_positions, labled_positions

In [10]:
# Input data as a multiline string, including labels
NaNNO_prim_positions = """
Na  0.5000000000000000  0.5000000000000000  0.0000000000000000
Nd  0.5000000000000000  0.5000000000000000  0.5000000000000000
Nb  0.0000000000000000  0.0000000000000000  0.2935347165073026
Nb  0.0000000000000000  0.0000000000000000  0.7064652834926974
O  0.0000000000000000  0.0000000000000000  0.5000000000000000
O  0.5000000000000000  0.0000000000000000  0.6645394444672732
O  0.0000000000000000  0.5000000000000000  0.6645394444672732
O  0.0000000000000000  0.5000000000000000  0.3354605255327243
O  0.5000000000000000  0.0000000000000000  0.3354605255327243
O  0.0000000000000000  0.0000000000000000  0.1299886821043827
O  0.0000000000000000  0.0000000000000000  0.8700113178956173
"""

NaNNO_super_unlabled_positions, NaNNO_super_labled_positions = expand_positions(NaNNO_prim_positions)

Unique Labels: dict_keys(['Na', 'Nd', 'Nb', 'O'])
0.5000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.5000000000000000 0.0000000000000000
0.5000000000000000 0.0000000000000000 0.5000000000000000
0.0000000000000000 0.5000000000000000 0.5000000000000000
0.5000000000000000 0.0000000000000000 0.2500000000000000
0.0000000000000000 0.5000000000000000 0.2500000000000000
0.5000000000000000 0.0000000000000000 0.7500000000000000
0.0000000000000000 0.5000000000000000 0.7500000000000000
0.0000000000000000 0.0000000000000000 0.1467673582536513
0.5000000000000000 0.5000000000000000 0.1467673582536513
0.0000000000000000 0.0000000000000000 0.6467673582536513
0.5000000000000000 0.5000000000000000 0.6467673582536513
0.0000000000000000 0.0000000000000000 0.3532326417463487
0.5000000000000000 0.5000000000000000 0.3532326417463487
0.0000000000000000 0.0000000000000000 0.8532326417463487
0.5000000000000000 0.5000000000000000 0.8532326417463487
0.0000000000000000 0.0000000000000000 

In [11]:
NaNNO_RX_unlabled_positions, NaNNO_RX_labled_positions = RXexpand_positions(NaNNO_prim_positions)

Unique Labels: dict_keys(['Na', 'Nd', 'Nb', 'O'])
0.2500000000000000 0.2500000000000000 0.0000000000000000
0.7500000000000000 0.2500000000000000 0.0000000000000000
0.2500000000000000 0.7500000000000000 0.0000000000000000
0.7500000000000000 0.7500000000000000 0.0000000000000000
0.2500000000000000 0.2500000000000000 0.5000000000000000
0.7500000000000000 0.2500000000000000 0.5000000000000000
0.2500000000000000 0.7500000000000000 0.5000000000000000
0.7500000000000000 0.7500000000000000 0.5000000000000000
0.2500000000000000 0.2500000000000000 0.2500000000000000
0.7500000000000000 0.2500000000000000 0.2500000000000000
0.2500000000000000 0.7500000000000000 0.2500000000000000
0.7500000000000000 0.7500000000000000 0.2500000000000000
0.2500000000000000 0.2500000000000000 0.7500000000000000
0.7500000000000000 0.2500000000000000 0.7500000000000000
0.2500000000000000 0.7500000000000000 0.7500000000000000
0.7500000000000000 0.7500000000000000 0.7500000000000000
0.0000000000000000 0.0000000000000000 

In [12]:
NHNNO_prim_positions = '''
Nd  0.5000000000000000  0.5000000000000000  0.5000000000000000
Nb  0.0000000000000000  0.0000000000000000  0.2973069247387983
Nb  0.0000000000000000  0.0000000000000000  0.7026930752612017
O  0.0000000000000000  0.0000000000000000  0.5000000000000000
O  0.5000000000000000  0.0000000000000000  0.6618823177966391
O  0.0000000000000000  0.5000000000000000  0.6618823177966391
O  0.0000000000000000  0.5000000000000000  0.3381176522033584
O  0.5000000000000000  0.0000000000000000  0.3381176522033584
O  0.0000000000000000  0.0000000000000000  0.1356209619514246
O  0.0000000000000000  0.0000000000000000  0.8643790380485754
N  0.5000000000000000  0.5000000000000000  0.0000000000000000
H  0.6601132673751522  0.6601132673751522  0.0529550833074524
H  0.3398864946248494  0.3398867626248361  0.0529550833074524
H  0.3398867626248361  0.6601135053751506  0.9470449136925438
H  0.6601132673751522  0.3398864946248494  0.9470444366925435
'''
NHNNO_RX_unlabled_positions, NHNNO_RX_labled_positions= RXexpand_positions(NHNNO_prim_positions)

Unique Labels: dict_keys(['Nd', 'Nb', 'O', 'N', 'H'])
0.2500000000000000 0.2500000000000000 0.2500000000000000
0.7500000000000000 0.2500000000000000 0.2500000000000000
0.2500000000000000 0.7500000000000000 0.2500000000000000
0.7500000000000000 0.7500000000000000 0.2500000000000000
0.2500000000000000 0.2500000000000000 0.7500000000000000
0.7500000000000000 0.2500000000000000 0.7500000000000000
0.2500000000000000 0.7500000000000000 0.7500000000000000
0.7500000000000000 0.7500000000000000 0.7500000000000000
0.0000000000000000 0.0000000000000000 0.1486534623693991
0.5000000000000000 0.0000000000000000 0.1486534623693991
0.0000000000000000 0.5000000000000000 0.1486534623693991
0.5000000000000000 0.5000000000000000 0.1486534623693991
0.0000000000000000 0.0000000000000000 0.6486534623693991
0.5000000000000000 0.0000000000000000 0.6486534623693991
0.0000000000000000 0.5000000000000000 0.6486534623693991
0.5000000000000000 0.5000000000000000 0.6486534623693991
0.0000000000000000 0.0000000000000

In [13]:
NHNNO_M5= '''   
  0.5115147631851542  0.9884855281148432  0.2500000000000000
  0.9884852368148458  0.5115144718851568  0.2500000000000000
  0.5115147631851542  0.9884855281148432  0.7500000000000000
  0.9884852368148458  0.5115144718851568  0.7500000000000000
  0.0045682836425129  0.0019421582897792  0.1496758583448283
  0.4954317163574871  0.4980578417102208  0.1496758583448283
  0.0045682836425129  0.0019421582897792  0.6496758583448283
  0.4954317163574871  0.4980578417102208  0.6496758583448283
  0.0019422291897797  0.0045681932425126  0.3503241416551717
  0.4980577708102203  0.4954318067574874  0.3503241416551717
  0.0019422291897797  0.0045681932425126  0.8503241416551717
  0.4980577708102203  0.4954318067574874  0.8503241416551717
  0.9425061666073447  0.9425061870073463  0.2500000000000000
  0.5574938333926553  0.5574938129926537  0.2500000000000000
  0.9425061666073447  0.9425061870073463  0.7500000000000000
  0.5574938333926553  0.5574938129926537  0.7500000000000000
  0.2500000000000000  0.2500000000000000  0.3173562518826643
  0.7500000000000000  0.7500000000000000  0.3519303673757861
  0.2500000000000000  0.2500000000000000  0.8173562518826643
  0.7500000000000000  0.7500000000000000  0.8519303673757861
  0.2500000000000000  0.7500000000000000  0.3308788866235233
  0.7500000000000000  0.2500000000000000  0.3274543786795334
  0.2500000000000000  0.7500000000000000  0.8308788866235233
  0.7500000000000000  0.2500000000000000  0.8274543786795334
  0.2500000000000000  0.7500000000000000  0.1725456950204602
  0.7500000000000000  0.2500000000000000  0.1691210096764735
  0.2500000000000000  0.7500000000000000  0.6725456950204602
  0.7500000000000000  0.2500000000000000  0.6691210096764735
  0.2500000000000000  0.2500000000000000  0.1826435810173308
  0.7500000000000000  0.7500000000000000  0.1480697697242093
  0.2500000000000000  0.2500000000000000  0.6826435810173308
  0.7500000000000000  0.7500000000000000  0.6480697697242093
  0.0615981510397106  0.0327044043425317  0.0704324538687757
  0.4384018489602894  0.4672955956574683  0.0704324538687757
  0.0615981510397106  0.0327044043425317  0.5704324538687757
  0.4384018489602894  0.4672955956574683  0.5704324538687757
  0.0327042648425291  0.0615988994397156  0.4295675461312243
  0.4672957351574709  0.4384011005602844  0.4295675461312243
  0.0327042648425291  0.0615988994397156  0.9295675461312243
  0.4672957351574709  0.4384011005602844  0.9295675461312243
  0.4795035712347442  0.0204971332652519  0.0000000000000000
  0.0204964287652558  0.4795028667347481  0.0000000000000000
  0.4795035712347442  0.0204971332652519  0.5000000000000000
  0.0204964287652558  0.4795028667347481  0.5000000000000000
  0.6352791982051258  0.0209103986452561  0.0266803689215109
  0.1813957292736319  0.4778978370610574  0.0276619378462755
  0.6352791982051258  0.0209103986452561  0.5266803689215038
  0.1813957292736319  0.4778978370610574  0.5276619378462755
  0.3186041667263595  0.0221020289389529  0.0276619378462755
  0.8647206977948656  0.4790894673547541  0.0266803689215109
  0.3186041667263595  0.0221020289389529  0.5276619378462755
  0.8647206977948656  0.4790894673547541  0.5266803689215038
  0.4790901683547304  0.8647213779948686  0.4733195490784752
  0.0221018639389357  0.3186034865263494  0.4723381411537204
  0.4790901683547304  0.8647213779948686  0.9733195490784894
  0.0221018639389357  0.3186034865263494  0.9723381411537204
  0.4778981510610549  0.1813965284736341  0.4723379026537273
  0.0209098466452602  0.6352786370051220  0.4733193105784821
  0.4778981510610549  0.1813965284736341  0.9723379026537273
  0.0209098466452602  0.6352786370051220  0.9733193105784963
'''
NHNNO_M5_unlabled = string_to_matrix(NHNNO_M5)

#Adding labeling matrix
Nd_array = np.array(['Nd'] * 4)
Nb_array = np.array(['Nb'] * 8)
O_array = np.array(['O'] * 28)
N_array = np.array(['N'] * 4)
H_array = np.array(['H'] * 16)

# Concatenate the arrays to form the final vector
vector = np.concatenate((Nd_array, Nb_array, O_array, N_array, H_array))
# Reshape the vector to be a column vector
vector_column = vector.reshape(-1, 1)

# Add the vector as the first column of the matrix
NHNNO_M5_labled = np.hstack((vector_column, NHNNO_M5_unlabled))

In [14]:
NHNNO_super_unlabled_positions = '''
  0.5000000000000000  0.0000000000000000  0.2500000000000000
  0.0000000000000000  0.5000000000000000  0.2500000000000000
  0.5000000000000000  0.0000000000000000  0.7500000000000000
  0.0000000000000000  0.5000000000000000  0.7500000000000000
  0.0000000000000000  0.0000000000000000  0.1486501333029082
  0.5000000000000000  0.5000000000000000  0.1486501333029082
  0.0000000000000000  0.0000000000000000  0.6486501333029082
  0.5000000000000000  0.5000000000000000  0.6486501333029082
  0.0000000000000000  0.0000000000000000  0.3513498666970918
  0.5000000000000000  0.5000000000000000  0.3513498666970918
  0.0000000000000000  0.0000000000000000  0.8513498666970918
  0.5000000000000000  0.5000000000000000  0.8513498666970918
  0.0000000000000000  0.0000000000000000  0.2500000000000000
  0.5000000000000000  0.5000000000000000  0.2500000000000000
  0.0000000000000000  0.0000000000000000  0.7500000000000000
  0.5000000000000000  0.5000000000000000  0.7500000000000000
  0.2500000000000000  0.2500000000000000  0.3309407311884414
  0.7500000000000000  0.7500000000000000  0.3309407311884414
  0.2500000000000000  0.2500000000000000  0.8309407311884414
  0.7500000000000000  0.7500000000000000  0.8309407311884414
  0.2500000000000000  0.7500000000000000  0.3309407311884414
  0.7500000000000000  0.2500000000000000  0.3309407311884414
  0.2500000000000000  0.7500000000000000  0.8309407311884414
  0.7500000000000000  0.2500000000000000  0.8309407311884414
  0.2500000000000000  0.7500000000000000  0.1690592538115538
  0.7500000000000000  0.2500000000000000  0.1690592538115538
  0.2500000000000000  0.7500000000000000  0.6690592538115538
  0.7500000000000000  0.2500000000000000  0.6690592538115538
  0.2500000000000000  0.2500000000000000  0.1690592538115538
  0.7500000000000000  0.7500000000000000  0.1690592538115538
  0.2500000000000000  0.2500000000000000  0.6690592538115538
  0.7500000000000000  0.7500000000000000  0.6690592538115538
  0.0000000000000000  0.0000000000000000  0.0678041001726299
  0.5000000000000000  0.5000000000000000  0.0678041001726299
  0.0000000000000000  0.0000000000000000  0.5678041001726299
  0.5000000000000000  0.5000000000000000  0.5678041001726299
  0.0000000000000000  0.0000000000000000  0.4321958998273701
  0.5000000000000000  0.5000000000000000  0.4321958998273701
  0.0000000000000000  0.0000000000000000  0.9321958998273701
  0.5000000000000000  0.5000000000000000  0.9321958998273701
  0.5000000000000000  0.0000000000000000  0.0000000000000000
  0.0000000000000000  0.5000000000000000  0.0000000000000000
  0.5000000000000000  0.0000000000000000  0.5000000000000000
  0.0000000000000000  0.5000000000000000  0.5000000000000000
  0.6601231254133708  0.0000000000000000  0.0264789663931921
  0.1601231254133708  0.5000000000000000  0.0264789663931921
  0.6601231254133708  0.0000000000000000  0.5264789663931921
  0.1601231254133708  0.5000000000000000  0.5264789663931921
  0.3398767705866206  0.9999998660000102  0.0264789663931921
  0.8398767705866206  0.4999998660000102  0.0264789663931921
  0.3398767705866206  0.9999998660000102  0.5264789663931921
  0.8398767705866206  0.4999998660000102  0.5264789663931921
  0.5000001339999898  0.8398767705866206  0.4735210321068024
  0.0000001339999898  0.3398767705866206  0.4735210321068024
  0.5000001339999898  0.8398767705866206  0.9735210321068024
  0.0000001339999898  0.3398767705866206  0.9735210321068024
  0.4999998810000008  0.1601232444133700  0.4735207936068093
  0.9999998810000008  0.6601232444133700  0.4735207936068093
  0.4999998810000008  0.1601232444133700  0.9735207936068093
  0.9999998810000008  0.6601232444133700  0.9735207936068093
'''
NHNNO_super_unlabled_positions = string_to_matrix(NHNNO_super_unlabled_positions)

#Adding labeling matrix
Nd_array = np.array(['Nd'] * 4)
Nb_array = np.array(['Nb'] * 8)
O_array = np.array(['O'] * 28)
N_array = np.array(['N'] * 4)
H_array = np.array(['H'] * 16)

# Concatenate the arrays to form the final vector
vector = np.concatenate((Nd_array, Nb_array, O_array, N_array, H_array))
# Reshape the vector to be a column vector
vector_column = vector.reshape(-1, 1)

# Add the vector as the first column of the matrix
NHNNO_super_labled_positions = np.hstack((vector_column, NHNNO_super_unlabled_positions))


NHNNO_RX_unlabled_positions = ''' 
  0.2500000000000000  0.2500000000000000  0.2500000000000000
  0.7500000000000000  0.2500000000000000  0.2500000000000000
  0.2500000000000000  0.7500000000000000  0.2500000000000000
  0.7500000000000000  0.7500000000000000  0.2500000000000000
  0.2500000000000000  0.2500000000000000  0.7500000000000000
  0.7500000000000000  0.2500000000000000  0.7500000000000000
  0.2500000000000000  0.7500000000000000  0.7500000000000000
  0.7500000000000000  0.7500000000000000  0.7500000000000000
  0.0000000000000000  0.0000000000000000  0.1486516294796871
  0.5000000000000000  0.0000000000000000  0.1486516294796871
  0.0000000000000000  0.5000000000000000  0.1486516294796871
  0.5000000000000000  0.5000000000000000  0.1486516294796871
  0.0000000000000000  0.0000000000000000  0.6486516294796871
  0.5000000000000000  0.0000000000000000  0.6486516294796871
  0.0000000000000000  0.5000000000000000  0.6486516294796871
  0.5000000000000000  0.5000000000000000  0.6486516294796871
  0.0000000000000000  0.0000000000000000  0.3513483705203129
  0.5000000000000000  0.0000000000000000  0.3513483705203129
  0.0000000000000000  0.5000000000000000  0.3513483705203129
  0.5000000000000000  0.5000000000000000  0.3513483705203129
  0.0000000000000000  0.0000000000000000  0.8513483705203129
  0.5000000000000000  0.0000000000000000  0.8513483705203129
  0.0000000000000000  0.5000000000000000  0.8513483705203129
  0.5000000000000000  0.5000000000000000  0.8513483705203129
  0.0000000000000000  0.0000000000000000  0.2500000000000000
  0.5000000000000000  0.0000000000000000  0.2500000000000000
  0.0000000000000000  0.5000000000000000  0.2500000000000000
  0.5000000000000000  0.5000000000000000  0.2500000000000000
  0.0000000000000000  0.0000000000000000  0.7500000000000000
  0.5000000000000000  0.0000000000000000  0.7500000000000000
  0.0000000000000000  0.5000000000000000  0.7500000000000000
  0.5000000000000000  0.5000000000000000  0.7500000000000000
  0.2500000000000000  0.0000000000000000  0.3309407168361815
  0.7500000000000000  0.0000000000000000  0.3309407168361815
  0.2500000000000000  0.5000000000000000  0.3309407168361815
  0.7500000000000000  0.5000000000000000  0.3309407168361815
  0.2500000000000000  0.0000000000000000  0.8309407168361815
  0.7500000000000000  0.0000000000000000  0.8309407168361815
  0.2500000000000000  0.5000000000000000  0.8309407168361815
  0.7500000000000000  0.5000000000000000  0.8309407168361815
  0.0000000000000000  0.2500000000000000  0.3309407168361815
  0.5000000000000000  0.2500000000000000  0.3309407168361815
  0.0000000000000000  0.7500000000000000  0.3309407168361815
  0.5000000000000000  0.7500000000000000  0.3309407168361815
  0.0000000000000000  0.2500000000000000  0.8309407168361815
  0.5000000000000000  0.2500000000000000  0.8309407168361815
  0.0000000000000000  0.7500000000000000  0.8309407168361815
  0.5000000000000000  0.7500000000000000  0.8309407168361815
  0.0000000000000000  0.2500000000000000  0.1690592681638137
  0.5000000000000000  0.2500000000000000  0.1690592681638137
  0.0000000000000000  0.7500000000000000  0.1690592681638137
  0.5000000000000000  0.7500000000000000  0.1690592681638137
  0.0000000000000000  0.2500000000000000  0.6690592681638137
  0.5000000000000000  0.2500000000000000  0.6690592681638137
  0.0000000000000000  0.7500000000000000  0.6690592681638137
  0.5000000000000000  0.7500000000000000  0.6690592681638137
  0.2500000000000000  0.0000000000000000  0.1690592681638137
  0.7500000000000000  0.0000000000000000  0.1690592681638137
  0.2500000000000000  0.5000000000000000  0.1690592681638137
  0.7500000000000000  0.5000000000000000  0.1690592681638137
  0.2500000000000000  0.0000000000000000  0.6690592681638137
  0.7500000000000000  0.0000000000000000  0.6690592681638137
  0.2500000000000000  0.5000000000000000  0.6690592681638137
  0.7500000000000000  0.5000000000000000  0.6690592681638137
  0.0000000000000000  0.0000000000000000  0.0678058900538048
  0.5000000000000000  0.0000000000000000  0.0678058900538048
  0.0000000000000000  0.5000000000000000  0.0678058900538048
  0.5000000000000000  0.5000000000000000  0.0678058900538048
  0.0000000000000000  0.0000000000000000  0.5678058900538048
  0.5000000000000000  0.0000000000000000  0.5678058900538048
  0.0000000000000000  0.5000000000000000  0.5678058900538048
  0.5000000000000000  0.5000000000000000  0.5678058900538048
  0.0000000000000000  0.0000000000000000  0.4321941099461952
  0.5000000000000000  0.0000000000000000  0.4321941099461952
  0.0000000000000000  0.5000000000000000  0.4321941099461952
  0.5000000000000000  0.5000000000000000  0.4321941099461952
  0.0000000000000000  0.0000000000000000  0.9321941099461952
  0.5000000000000000  0.0000000000000000  0.9321941099461952
  0.0000000000000000  0.5000000000000000  0.9321941099461952
  0.5000000000000000  0.5000000000000000  0.9321941099461952
  0.2500000000000000  0.2500000000000000  0.0000000000000000
  0.7500000000000000  0.2500000000000000  0.0000000000000000
  0.2500000000000000  0.7500000000000000  0.0000000000000000
  0.7500000000000000  0.7500000000000000  0.0000000000000000
  0.2500000000000000  0.2500000000000000  0.5000000000000000
  0.7500000000000000  0.2500000000000000  0.5000000000000000
  0.2500000000000000  0.7500000000000000  0.5000000000000000
  0.7500000000000000  0.7500000000000000  0.5000000000000000
  0.3300600300334793  0.3300600300334793  0.0264786194285023
  0.8300600300334793  0.3300600300334793  0.0264786194285023
  0.3300600300334793  0.8300600300334793  0.0264786194285023
  0.8300600300334793  0.8300600300334793  0.0264786194285023
  0.3300600300334793  0.3300600300334793  0.5264786194285023
  0.8300600300334793  0.3300600300334793  0.5264786194285023
  0.3300600300334793  0.8300600300334793  0.5264786194285023
  0.8300600300334793  0.8300600300334793  0.5264786194285023
  0.1699398509665215  0.1699399849665113  0.0264786194285023
  0.6699398509665215  0.1699399849665113  0.0264786194285023
  0.1699398509665215  0.6699399849665184  0.0264786194285023
  0.6699398509665215  0.6699399849665184  0.0264786194285023
  0.1699398509665215  0.1699399849665113  0.5264786194285023
  0.6699398509665215  0.1699399849665113  0.5264786194285023
  0.1699398509665215  0.6699399849665184  0.5264786194285023
  0.6699398509665215  0.6699399849665184  0.5264786194285023
  0.1699399849665113  0.3300601490334785  0.4735213790714923
  0.6699399849665184  0.3300601490334785  0.4735213790714923
  0.1699399849665113  0.8300601490334785  0.4735213790714923
  0.6699399849665184  0.8300601490334785  0.4735213790714923
  0.1699399849665113  0.3300601490334785  0.9735213790714923
  0.6699399849665184  0.3300601490334785  0.9735213790714923
  0.1699399849665113  0.8300601490334785  0.9735213790714923
  0.6699399849665184  0.8300601490334785  0.9735213790714923
  0.3300600300334793  0.1699398509665215  0.4735211405714921
  0.8300600300334793  0.1699398509665215  0.4735211405714921
  0.3300600300334793  0.6699398509665215  0.4735211405714921
  0.8300600300334793  0.6699398509665215  0.4735211405714921
  0.3300600300334793  0.1699398509665215  0.9735211405714992
  0.8300600300334793  0.1699398509665215  0.9735211405714992
  0.3300600300334793  0.6699398509665215  0.9735211405714992
  0.8300600300334793  0.6699398509665215  0.9735211405714992
'''
NHNNO_RX_unlabled_positions = string_to_matrix(NHNNO_RX_unlabled_positions)

#Adding labeling matrix
Nd_array = np.array(['Nd'] * 8)
Nb_array = np.array(['Nb'] * 16)
O_array = np.array(['O'] * 56)
N_array = np.array(['N'] * 8)
H_array = np.array(['H'] * 32)

# Concatenate the arrays to form the final vector
vector = np.concatenate((Nd_array, Nb_array, O_array, N_array, H_array))
# Reshape the vector to be a column vector
vector_column = vector.reshape(-1, 1)

# Add the vector as the first column of the matrix
NHNNO_RX_labled_positions = np.hstack((vector_column, NHNNO_RX_unlabled_positions))



In [15]:
NHNNO_super_lattice = '''
     5.4293434508814471    0.0000000000000000    0.0000000000000000
     0.0000000000000000    5.4293434508814471    0.0000000000000000
     0.0000000000000000    0.0000000000000000   22.1542825860808676
'''
NHNNO_super_lattice = string_to_matrix(NHNNO_super_lattice)

In [16]:
NaNNO_RX_unlabled_positions = '''
  0.2500000000000000  0.2500000000000000  0.0000000000000000
  0.7500000000000000  0.2500000000000000  0.0000000000000000
  0.2500000000000000  0.7500000000000000  0.0000000000000000
  0.7500000000000000  0.7500000000000000  0.0000000000000000
  0.2500000000000000  0.2500000000000000  0.5000000000000000
  0.7500000000000000  0.2500000000000000  0.5000000000000000
  0.2500000000000000  0.7500000000000000  0.5000000000000000
  0.7500000000000000  0.7500000000000000  0.5000000000000000
  0.2500000000000000  0.2500000000000000  0.2500000000000000
  0.7500000000000000  0.2500000000000000  0.2500000000000000
  0.2500000000000000  0.7500000000000000  0.2500000000000000
  0.7500000000000000  0.7500000000000000  0.2500000000000000
  0.2500000000000000  0.2500000000000000  0.7500000000000000
  0.7500000000000000  0.2500000000000000  0.7500000000000000
  0.2500000000000000  0.7500000000000000  0.7500000000000000
  0.7500000000000000  0.7500000000000000  0.7500000000000000
  0.0000000000000000  0.0000000000000000  0.1467680514095733
  0.5000000000000000  0.0000000000000000  0.1467680514095733
  0.0000000000000000  0.5000000000000000  0.1467680514095733
  0.5000000000000000  0.5000000000000000  0.1467680514095733
  0.0000000000000000  0.0000000000000000  0.6467680514095733
  0.5000000000000000  0.0000000000000000  0.6467680514095733
  0.0000000000000000  0.5000000000000000  0.6467680514095733
  0.5000000000000000  0.5000000000000000  0.6467680514095733
  0.0000000000000000  0.0000000000000000  0.3532319485904267
  0.5000000000000000  0.0000000000000000  0.3532319485904267
  0.0000000000000000  0.5000000000000000  0.3532319485904267
  0.5000000000000000  0.5000000000000000  0.3532319485904267
  0.0000000000000000  0.0000000000000000  0.8532319485904267
  0.5000000000000000  0.0000000000000000  0.8532319485904267
  0.0000000000000000  0.5000000000000000  0.8532319485904267
  0.5000000000000000  0.5000000000000000  0.8532319485904267
  0.0000000000000000  0.0000000000000000  0.2500000000000000
  0.5000000000000000  0.0000000000000000  0.2500000000000000
  0.0000000000000000  0.5000000000000000  0.2500000000000000
  0.5000000000000000  0.5000000000000000  0.2500000000000000
  0.0000000000000000  0.0000000000000000  0.7500000000000000
  0.5000000000000000  0.0000000000000000  0.7500000000000000
  0.0000000000000000  0.5000000000000000  0.7500000000000000
  0.5000000000000000  0.5000000000000000  0.7500000000000000
  0.2500000000000000  0.0000000000000000  0.3322695969504821
  0.7500000000000000  0.0000000000000000  0.3322695969504821
  0.2500000000000000  0.5000000000000000  0.3322695969504821
  0.7500000000000000  0.5000000000000000  0.3322695969504821
  0.2500000000000000  0.0000000000000000  0.8322695969504821
  0.7500000000000000  0.0000000000000000  0.8322695969504821
  0.2500000000000000  0.5000000000000000  0.8322695969504821
  0.7500000000000000  0.5000000000000000  0.8322695969504821
  0.0000000000000000  0.2500000000000000  0.3322695969504821
  0.5000000000000000  0.2500000000000000  0.3322695969504821
  0.0000000000000000  0.7500000000000000  0.3322695969504821
  0.5000000000000000  0.7500000000000000  0.3322695969504821
  0.0000000000000000  0.2500000000000000  0.8322695969504821
  0.5000000000000000  0.2500000000000000  0.8322695969504821
  0.0000000000000000  0.7500000000000000  0.8322695969504821
  0.5000000000000000  0.7500000000000000  0.8322695969504821
  0.0000000000000000  0.2500000000000000  0.1677303880495131
  0.5000000000000000  0.2500000000000000  0.1677303880495131
  0.0000000000000000  0.7500000000000000  0.1677303880495131
  0.5000000000000000  0.7500000000000000  0.1677303880495131
  0.0000000000000000  0.2500000000000000  0.6677303880495131
  0.5000000000000000  0.2500000000000000  0.6677303880495131
  0.0000000000000000  0.7500000000000000  0.6677303880495131
  0.5000000000000000  0.7500000000000000  0.6677303880495131
  0.2500000000000000  0.0000000000000000  0.1677303880495131
  0.7500000000000000  0.0000000000000000  0.1677303880495131
  0.2500000000000000  0.5000000000000000  0.1677303880495131
  0.7500000000000000  0.5000000000000000  0.1677303880495131
  0.2500000000000000  0.0000000000000000  0.6677303880495131
  0.7500000000000000  0.0000000000000000  0.6677303880495131
  0.2500000000000000  0.5000000000000000  0.6677303880495131
  0.7500000000000000  0.5000000000000000  0.6677303880495131
  0.0000000000000000  0.0000000000000000  0.0649933717505888
  0.5000000000000000  0.0000000000000000  0.0649933717505888
  0.0000000000000000  0.5000000000000000  0.0649933717505888
  0.5000000000000000  0.5000000000000000  0.0649933717505888
  0.0000000000000000  0.0000000000000000  0.5649933717505888
  0.5000000000000000  0.0000000000000000  0.5649933717505888
  0.0000000000000000  0.5000000000000000  0.5649933717505888
  0.5000000000000000  0.5000000000000000  0.5649933717505888
  0.0000000000000000  0.0000000000000000  0.4350066282494112
  0.5000000000000000  0.0000000000000000  0.4350066282494112
  0.0000000000000000  0.5000000000000000  0.4350066282494112
  0.5000000000000000  0.5000000000000000  0.4350066282494112
  0.0000000000000000  0.0000000000000000  0.9350066282494112
  0.5000000000000000  0.0000000000000000  0.9350066282494112
  0.0000000000000000  0.5000000000000000  0.9350066282494112
  0.5000000000000000  0.5000000000000000  0.9350066282494112 
'''
NaNNO_RX_unlabled_positions = string_to_matrix(NaNNO_RX_unlabled_positions)

NaNNO_super_unlabled_positions = '''
  0.5000000000000000  0.0000000000000000  0.0000000000000000
  0.0000000000000000  0.5000000000000000  0.0000000000000000
  0.5000000000000000  0.0000000000000000  0.5000000000000000
  0.0000000000000000  0.5000000000000000  0.5000000000000000
  0.5000000000000000  0.0000000000000000  0.2500000000000000
  0.0000000000000000  0.5000000000000000  0.2500000000000000
  0.5000000000000000  0.0000000000000000  0.7500000000000000
  0.0000000000000000  0.5000000000000000  0.7500000000000000
  0.0000000000000000  0.0000000000000000  0.1467684323133298
  0.5000000000000000  0.5000000000000000  0.1467684323133298
  0.0000000000000000  0.0000000000000000  0.6467684323133298
  0.5000000000000000  0.5000000000000000  0.6467684323133298
  0.0000000000000000  0.0000000000000000  0.3532315676866702
  0.5000000000000000  0.5000000000000000  0.3532315676866702
  0.0000000000000000  0.0000000000000000  0.8532315676866702
  0.5000000000000000  0.5000000000000000  0.8532315676866702
  0.0000000000000000  0.0000000000000000  0.2500000000000000
  0.5000000000000000  0.5000000000000000  0.2500000000000000
  0.0000000000000000  0.0000000000000000  0.7500000000000000
  0.5000000000000000  0.5000000000000000  0.7500000000000000
  0.2500000000000000  0.2500000000000000  0.3322694389203988
  0.7500000000000000  0.7500000000000000  0.3322694389203988
  0.2500000000000000  0.2500000000000000  0.8322694389203988
  0.7500000000000000  0.7500000000000000  0.8322694389203988
  0.2500000000000000  0.7500000000000000  0.3322694389203988
  0.7500000000000000  0.2500000000000000  0.3322694389203988
  0.2500000000000000  0.7500000000000000  0.8322694389203988
  0.7500000000000000  0.2500000000000000  0.8322694389203988
  0.2500000000000000  0.7500000000000000  0.1677305460795964
  0.7500000000000000  0.2500000000000000  0.1677305460795964
  0.2500000000000000  0.7500000000000000  0.6677305460795964
  0.7500000000000000  0.2500000000000000  0.6677305460795964
  0.2500000000000000  0.2500000000000000  0.1677305460795964
  0.7500000000000000  0.7500000000000000  0.1677305460795964
  0.2500000000000000  0.2500000000000000  0.6677305460795964
  0.7500000000000000  0.7500000000000000  0.6677305460795964
  0.0000000000000000  0.0000000000000000  0.0649935203256575
  0.5000000000000000  0.5000000000000000  0.0649935203256575
  0.0000000000000000  0.0000000000000000  0.5649935203256575
  0.5000000000000000  0.5000000000000000  0.5649935203256575
  0.0000000000000000  0.0000000000000000  0.4350064796743425
  0.5000000000000000  0.5000000000000000  0.4350064796743425
  0.0000000000000000  0.0000000000000000  0.9350064796743425
  0.5000000000000000  0.5000000000000000  0.9350064796743425
'''
NaNNO_super_unlabled_positions =  string_to_matrix(NaNNO_super_unlabled_positions)

#Adding labeling matrix
Na_array = np.array(['Na'] * 4)
Nd_array = np.array(['Nd'] * 4)
Nb_array = np.array(['Nb'] * 8)
O_array = np.array(['O'] * 28)

# Concatenate the arrays to form the final vector
vector = np.concatenate((Na_array, Nd_array, Nb_array, O_array))
# Reshape the vector to be a column vector
vector_column = vector.reshape(-1, 1)

# Add the vector as the first column of the matrix
NaNNO_super_labled_positions = np.hstack((vector_column, NaNNO_super_unlabled_positions))


#Adding labeling matrix
Na_array = np.array(['Na'] * 8)
Nd_array = np.array(['Nd'] * 8)
Nb_array = np.array(['Nb'] * 16)
O_array = np.array(['O'] * 56)

# Concatenate the arrays to form the final vector
vector = np.concatenate((Na_array, Nd_array, Nb_array, O_array))
# Reshape the vector to be a column vector
vector_column = vector.reshape(-1, 1)
NaNNO_RX_labled_positions = np.hstack((vector_column, NaNNO_RX_unlabled_positions))


In [17]:
NaNNO_super_lattice = '''
     5.4135959837361973    0.0000000000000000    0.0000000000000000
     0.0000000000000000    5.4135959837361973    0.0000000000000000
     0.0000000000000000    0.0000000000000000   21.8878385941033713
'''
NaNNO_super_lattice = string_to_matrix(NaNNO_super_lattice)

In [18]:
NaNNO_RX_lattice = '''  
     7.6560233118871910    0.0000000000000000    0.0000000000000000
     0.0000000000000000    7.6560233118871910    0.0000000000000000
     0.0000000000000000    0.0000000000000000   21.8878652958533770
'''
NaNNO_RX_lattice = string_to_matrix(NaNNO_RX_lattice)

In [19]:
NHNNO_RX_lattice = '''
     7.6783030983169080    0.0000000000000000    0.0000000000000000
     0.0000000000000000    7.6783030983169080    0.0000000000000000
     0.0000000000000000    0.0000000000000000   22.1545388808033330
'''
NHNNO_RX_lattice = string_to_matrix(NHNNO_RX_lattice)

### adding some random noise to make P1 structure

In [3]:
RNNO_Imma = '''
  0.0000000000000000  0.0000000000000000  0.2500000000000000
  0.5000000000000000  0.5000000000000000  0.2500000000000000
  0.0000000000000000  0.0000000000000000  0.7500000000000000
  0.5000000000000000  0.5000000000000000  0.7500000000000000
  0.0000000000000000  0.0100898540040006  0.0000000000000000
  0.5000000000000000  0.4899101459959994  0.0000000000000000
  0.0000000000000000  0.9899101459959994  0.5000000000000000
  0.5000000000000000  0.5100898540040006  0.5000000000000000
  0.5000000000000000  0.9950691346854086  0.1011216350744633
  0.0000000000000000  0.5049308653145914  0.1011216350744633
  0.5000000000000000  0.0049308653145914  0.6011216350744704
  0.0000000000000000  0.4950691346854086  0.6011216350744704
  0.5000000000000000  0.0049308653145914  0.3988783499255319
  0.0000000000000000  0.4950691346854086  0.3988783499255319
  0.5000000000000000  0.9950691346854086  0.8988783499255319
  0.0000000000000000  0.5049308653145914  0.8988783499255319
  0.2500000000000000  0.7500000000000000  0.0707203686650644
  0.7500000000000000  0.2500000000000000  0.0946361184131703
  0.2500000000000000  0.7500000000000000  0.5946361184131703
  0.7500000000000000  0.2500000000000000  0.5707203686650644
  0.2500000000000000  0.7500000000000000  0.4053638815868297
  0.7500000000000000  0.2500000000000000  0.4292796313349356
  0.2500000000000000  0.7500000000000000  0.9292796313349356
  0.7500000000000000  0.2500000000000000  0.9053638815868297
  0.2500000000000000  0.2500000000000000  0.0946361184131703
  0.7500000000000000  0.7500000000000000  0.0707203686650644
  0.2500000000000000  0.2500000000000000  0.5707203686650644
  0.7500000000000000  0.7500000000000000  0.5946361184131703
  0.2500000000000000  0.2500000000000000  0.4292796313349356
  0.7500000000000000  0.7500000000000000  0.4053638815868297
  0.2500000000000000  0.2500000000000000  0.9053638815868297
  0.7500000000000000  0.7500000000000000  0.9292796313349356
  0.5000000000000000  0.9376345942216702  0.1809127204779060
  0.0000000000000000  0.5623654057783298  0.1809127204779060
  0.5000000000000000  0.0623654057783298  0.6809127204779131
  0.0000000000000000  0.4376345942216702  0.6809127204779131
  0.5000000000000000  0.0623654057783298  0.3190872945220917
  0.0000000000000000  0.4376345942216702  0.3190872945220917
  0.5000000000000000  0.9376345942216702  0.8190872945220846
  0.0000000000000000  0.5623654057783298  0.8190872945220846
  0.5000000000000000  0.0708445720799631  0.0000000000000000
  0.0000000000000000  0.4291554279200369  0.0000000000000000
  0.5000000000000000  0.9291554279200369  0.5000000000000000
  0.0000000000000000  0.5708445720799631  0.5000000000000000'''
RNNO_Imma = string_to_matrix(RNNO_Imma)

In [5]:
N, M = RNNO_Imma.shape
random_array = np.random.rand(N, M)
small_numbers_array = random_array / 1e3  # You can adjust the divisor as needed
distorted_positions = RNNO_Imma + small_numbers_array

# Convert the matrix to a string in the desired format
labled_distorted_positions = matrix_to_string(distorted_positions)
print(labled_distorted_positions)

0.0005825942086429 0.0003872678743705 0.2500734467534397
0.5006013073587026 0.5004409772570467 0.2504379496812491
0.0004769153028027 0.0002549271925540 0.7501674746138008
0.5008195137849638 0.5004973143938509 0.7505825589361458
0.0002360292041719 0.0102251647965503 0.0007242647130318
0.5002884713235471 0.4901607886331018 0.0003180312372630
0.0004119520022746 0.9902203348718135 0.5007647747008550
0.5008632026088822 0.5109753967653526 0.5006785070276372
0.5001182087778605 0.9951406755812471 0.1014812256843377
0.0003656131645964 0.5057380200211202 0.1018374593521345
0.5008060980740516 0.0049862874429781 0.6019869882567693
0.0006383403217476 0.4953757349477024 0.6019712335042766
0.5000415146366024 0.0053426936857181 0.3993631567739236
0.0000873638934171 0.4957977374335535 0.3996559436246390
0.5002655514258738 0.9958167368935965 0.8998506332378582
0.0009913391949033 0.5049385099858731 0.8998542483848363
0.2504397418201609 0.7500798552116550 0.0707348886770447
0.7500204760307699 0.2507975490

## Add mode distortions from ISODISTORT


### Keeping direction of the mode the same, the direction can be changed now by adding subsequent modes. The direction of the eigenvector can also be subsequently rotated as desired.

In [21]:
import re

def select_modes_by_header(filename, pattern, columns_to_remove=None):
    with open(filename, 'r') as file:
        input_data = file.read()
    
    data_arrays = []
    pattern_regex = re.compile(pattern)
    
    # Split the input data into individual arrays
    arrays = input_data.strip().split('\n\n')

    for array in arrays:
        lines = array.split('\n')

        # Check if the header matches the pattern
        header = lines[0]
        if not pattern_regex.search(header):
            continue
        
        # Remove the first line (header)
        lines.pop(0)

        data = []
        for line in lines:
            if line.strip():  # Check if the line is not empty
                components = line.split()
                # Skip the first component and convert the rest to floats
                row = [components[0]] + [float(component) for component in components[1:]]
                
                data.append(row)

        # Remove specific columns if specified
        if columns_to_remove:
            for row in data:
                for col_index in sorted(columns_to_remove, reverse=True):
                    # Adjusted to account for the skipped first column
                    if col_index != 0:
                        del row[col_index]

        # Construct the data array dictionary
        data_arrays.append({'header': header, 'data': data})

    return data_arrays

def select_modes_by_header(filename, pattern, columns_to_remove=None):
    """ Extracts data sections from a file based on header matching a regex pattern.
    
    Args:
        filename (str): The path to the input file.
        pattern (str): Regex pattern to match in the headers.
        columns_to_remove (list of int, optional): Indices of columns to remove from the data, excluding the first column.
    
    Returns:
        list of dict: List of dictionaries containing the header and data of matching sections.
    """
    pattern_regex = re.compile(pattern)
    data_arrays = []
    
    try:
        with open(filename, 'r') as file:
            current_section = []
            include_section = False
            
            for line in file:
                line = line.strip()
                if not line:  # Empty line signifies end of a section
                    if include_section and current_section:
                        data_arrays.append(process_section(current_section, columns_to_remove))
                    current_section = []
                    include_section = False
                else:
                    if not current_section:  # This means we're at the header
                        include_section = bool(pattern_regex.search(line))
                    current_section.append(line)
                    
            # Handle the last section if file does not end with a newline
            if include_section and current_section:
                data_arrays.append(process_section(current_section, columns_to_remove))
    
    except IOError as e:
        print(f"Error reading file: {e}")
        return []
    
    return data_arrays

def process_section(section, columns_to_remove):
    """ Processes a section, removing specified columns and converting data to float. """
    header = section[0]
    data = []
    for line in section[1:]:
        components = line.split()
        if columns_to_remove:
            row = [components[i] for i in range(len(components)) if i not in columns_to_remove]
        else:
            row = components
        data.append([float(x) if i > 0 else x for i, x in enumerate(row)])  # Convert all but first component to float
    return {'header': header, 'data': data}

# Example usage
# result = select_modes_by_header('./ISODISTORT_NaNNO_2x2x2.txt', r'\]R3\+\(', [1,2,3])


In [28]:
def add_distortions_by_labels(matrix1, matrix2, lattice_basis):
    """
    Adds distortions to rows in matrix1 based on corresponding rows in matrix2.
    Args:
    - matrix1: Original positions to be distorted, labled with numbers to match each mode to each atom exactly
    - matrix2: SAM distortions with labels
    - lattice_basis: Lattice constants of the supercell
    Returns:
    - result_matrix: Matrix1 with distortions added
    """
    matrix_copy = np.copy(matrix1[:, 1:].astype(float))

    # Convert matrix2 to Cartesian coordinates
    cartesian_matrix2 = np.dot(matrix2[:, 1:].astype(float), lattice_basis)

    # Normalize matrix in cartesian: MODE AMOUNTS TO 1Ang
    normalized_matrix2 = cartesian_matrix2 / np.linalg.norm(cartesian_matrix2)

    inverse_t = np.linalg.inv(lattice_basis)
    
    #bring matrix back to direct
    direct_normalized = np.dot(normalized_matrix2,inverse_t)

    for row, normalized_row in zip(matrix2, direct_normalized):
        label = row[0]
        for i, row1 in enumerate(matrix1):
            if row1[0] == label:
                distortion = normalized_row * 0.01 #1mAng distortion
                matrix_copy[i] += distortion

    return np.hstack((matrix1[:, :1], matrix_copy))

def add_numbers2labels(data):
    data = np.array(data)
    # Dictionary to keep track of label counts
    label_counts = {}
    # Iterate through the array and append numbers to labels
    for i, row in enumerate(data):
        label = row[0]
        if label not in label_counts:
            label_counts[label] = 1
        else:
            label_counts[label] += 1
        data[i][0] = f"{label}{label_counts[label]}"
    return data


# Process the data with specified columns to remove
#result = select_modes_by_header('./ISODISTORT_NHNNO_r2xr2x2.txt', r'\]M5\(', [1,2,3])
#print(result)
#result = select_modes_by_header('./ISODISTORT_NaNNO_2x2x2.txt', r'\]X4\-\(', [1,2,3])
result = select_modes_by_header('../ISODISTORT/ISODISTORT_NHNNO_2x2x2.txt', r'\]M5\(', [1,2,3])
#print(result[-1]['data'])

for i in range(len(result)):
    moded_matrix = add_distortions_by_labels(np.array([row[:] for row in add_numbers2labels(NHNNO_RX_labled_positions)]), np.array(result[i]['data']), NHNNO_RX_lattice) #labled matrix had columns of strings - convert to float    

    formatted_unlabeled = matrix_to_string(moded_matrix[:,1:].astype(float))# + A5minus_dist*0.1)#+ GM5minus_eigenvec*0.1)# + A5minus_dist*0.1)
    print(formatted_unlabeled, '\n')
    #print(formatted_labeled)

0.2503255927732975 0.2496744072267025 0.2500000000000000
0.7496744072267024 0.2503255927732975 0.2500000000000000
0.2496744072267025 0.7503255927732976 0.2500000000000000
0.7503255927732976 0.7496744072267024 0.2500000000000000
0.2503255927732975 0.2496744072267025 0.7500000000000000
0.7496744072267024 0.2503255927732975 0.7500000000000000
0.2496744072267025 0.7503255927732976 0.7500000000000000
0.7503255927732976 0.7496744072267024 0.7500000000000000
0.0000000000000000 0.0000000000000000 0.1486516294796871
0.5000000000000000 0.0000000000000000 0.1486516294796871
0.0000000000000000 0.5000000000000000 0.1486516294796871
0.5000000000000000 0.5000000000000000 0.1486516294796871
0.0000000000000000 0.0000000000000000 0.6486516294796871
0.5000000000000000 0.0000000000000000 0.6486516294796871
0.0000000000000000 0.5000000000000000 0.6486516294796871
0.5000000000000000 0.5000000000000000 0.6486516294796871
0.0000000000000000 0.0000000000000000 0.3513483705203129
0.5000000000000000 0.0000000000

### Pure SAMs

In [29]:
def Make_SAMs(matrix1, matrix2, lattice_basis):

    # Convert matrix2 to Cartesian coordinates
    cartesian_matrix2 = np.dot(matrix2[:, 1:].astype(float), lattice_basis)

    # Normalize entire matrix2   
    normalized_matrix2 = cartesian_matrix2 / np.linalg.norm(cartesian_matrix2)#, axis=1)[:, np.newaxis]

    shape_matrix1 = matrix1[:, 1:].astype(float).shape
    # Create a new empty matrix of the same shape as matrix1
    empty_matrix_pos = np.zeros(shape_matrix1)

    for row, normalized_row in zip(matrix2, normalized_matrix2):
        label = row[0]
        for i, row1 in enumerate(matrix1):
            if row1[0] == label:
                empty_matrix_pos[i] += normalized_row
                
    return empty_matrix_pos #np.hstack((matrix1[:, :1], matrix_copy))

# Process the data with specified columns to remove
#result = select_modes_by_header('./ISODISTORT_NHNNO_r2xr2x2.txt', r']A5', [1,2,3])
SAM_modes = []
for i in range(len(result)):
    
    numbered_positions = np.array([row[:] for row in add_numbers2labels(NHNNO_RX_labled_positions)])

    ith_mode = np.array(result[i]['data'])

    moded_matrix = Make_SAMs(numbered_positions, ith_mode, NHNNO_RX_lattice) #labled matrix had columns of strings - convert to float    
    #    moded_matrix = add_distortions_by_labels(moded_matrix, np.array(result[i+1]['data']),lattice_basis)
    SAM_modes.append(moded_matrix)
    formatted_unlabeled = matrix_to_string(moded_matrix[:,:])
    print(formatted_unlabeled, '\n')

0.2500000000000001 -0.2500000000000001 0.0000000000000000
-0.2500000000000001 0.2500000000000001 0.0000000000000000
-0.2500000000000001 0.2500000000000001 0.0000000000000000
0.2500000000000001 -0.2500000000000001 0.0000000000000000
0.2500000000000001 -0.2500000000000001 0.0000000000000000
-0.2500000000000001 0.2500000000000001 0.0000000000000000
-0.2500000000000001 0.2500000000000001 0.0000000000000000
0.2500000000000001 -0.2500000000000001 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.0000000000000000 0.0000000000000000
0.0000000000000000 0.00

#### Make mode_masses

In [30]:
elements = {
    'Na': 22.989769,
    'Nd': 144.242,
    'Nb': 92.90638,
    'O': 15.9994
}
elements = {
    'Nd': 144.242,
    'Nb': 92.90638,
    'O': 15.9994,
    'N1': 14.0067,
    'H': 1
}
# Process the data with specified columns to remove
#result = select_modes_by_header('./ISODISTORT_NHNNO_r2xr2x2.txt', r']A5', [1,2,3])
mode_masses = []
# Iterate over each section of the data
for section in result:
    # Flatten the list of lists into a single list
    flattened_section = [element for sublist in section['data'] for element in sublist if isinstance(element, str)]
    
    # Check for each pattern in the current flattened section
    for pattern, mass in elements.items():
        if any(re.search(pattern, element) for element in flattened_section):
            mode_masses.append(mass)


for number in mode_masses:
    print(number)

144.242
144.242
92.90638
92.90638
92.90638
92.90638
15.9994
15.9994
15.9994
15.9994
15.9994
15.9994
15.9994
15.9994
15.9994
15.9994
14.0067
14.0067
1
1
1
1
1
1


## Combining eigenvec

In [25]:
M5_relax= '''   
  0.5099490896719487  0.0000000000000000  0.2502940620301288
  0.9900509103280513  0.5000000000000000  0.2502940620301288
  0.5099490896719487  0.0000000000000000  0.7502940620301288
  0.9900509103280513  0.5000000000000000  0.7502940620301288
  0.0061194981287969  0.0000000000000000  0.1494526672530441
  0.4938805018712031  0.5000000000000000  0.1494526672530441
  0.0061194981287969  0.0000000000000000  0.6494526672530441
  0.4938805018712031  0.5000000000000000  0.6494526672530441
  0.0038050005287147  0.0000000000000000  0.3511679404590140
  0.4961949994712853  0.5000000000000000  0.3511679404590140
  0.0038050005287147  0.0000000000000000  0.8511679404590140
  0.4961949994712853  0.5000000000000000  0.8511679404590140
  0.9302264234359470  0.0000000000000000  0.2493776488955604
  0.5697735765640530  0.5000000000000000  0.2493776488955604
  0.9302264234359470  0.0000000000000000  0.7493776488955604
  0.5697735765640530  0.5000000000000000  0.7493776488955604
  0.2500000000000000  0.2500000000000000  0.3214848533403654
  0.7500000000000000  0.7500000000000000  0.3425197245614768
  0.2500000000000000  0.2500000000000000  0.8214848533403654
  0.7500000000000000  0.7500000000000000  0.8425197245614768
  0.2500000000000000  0.7500000000000000  0.3214848533403654
  0.7500000000000000  0.2500000000000000  0.3425197245614768
  0.2500000000000000  0.7500000000000000  0.8214848533403654
  0.7500000000000000  0.2500000000000000  0.8425197245614768
  0.2500000000000000  0.7500000000000000  0.1807481094962426
  0.7500000000000000  0.2500000000000000  0.1544482527199165
  0.2500000000000000  0.7500000000000000  0.6807481094962498
  0.7500000000000000  0.2500000000000000  0.6544482527199165
  0.2500000000000000  0.2500000000000000  0.1807481094962498
  0.7500000000000000  0.7500000000000000  0.1544482527199165
  0.2500000000000000  0.2500000000000000  0.6807481094962498
  0.7500000000000000  0.7500000000000000  0.6544482527199165
  0.0830444897504705  0.0000000000000000  0.0709194818662553
  0.4169555102495295  0.5000000000000000  0.0709194818662553
  0.0830444897504705  0.0000000000000000  0.5709194818662553
  0.4169555102495295  0.5000000000000000  0.5709194818662553
  0.0493385202956915  0.0000000000000000  0.4310471274918513
  0.4506614797043085  0.5000000000000000  0.4310471274918513
  0.0493385202956915  0.0000000000000000  0.9310471274918513
  0.4506614797043085  0.5000000000000000  0.9310471274918513
  0.4792293280455056  0.0000000000000000  0.9994775705542693
  0.0207706719544944  0.5000000000000000  0.9994775705542693
  0.4792293280455056  0.0000000000000000  0.4994775705542693
  0.0207706719544944  0.5000000000000000  0.4994775705542693
  0.6368411496439137  0.0000000000000000  0.0250633096825794
  0.1801335101752954  0.5000000000000000  0.0284641117967368
  0.6368411496439137  0.0000000000000000  0.5250633096825794
  0.1801335101752954  0.5000000000000000  0.5284641117967368
  0.3198663858247031  0.9999998660000102  0.0284641117967368
  0.8631587463560777  0.4999998660000102  0.0250633096825794
  0.3198663858247031  0.9999998660000102  0.5284641117967368
  0.8631587463560777  0.4999998660000102  0.5250633096825794
  0.4738547852520867  0.8406153295864200  0.4727675534262517
  0.0261454827478929  0.3406153295864200  0.4727675534262517
  0.4738547852520867  0.8406153295864200  0.9727675534262517
  0.0261454827478929  0.3406153295864200  0.9727675534262517
  0.4738545322520977  0.1593846854135634  0.4727673149262586
  0.0261452297479039  0.6593846854135705  0.4727673149262586
  0.4738545322520977  0.1593846854135634  0.9727673149262586
  0.0261452297479039  0.6593846854135705  0.9727673149262586
'''

M5_relax = string_to_matrix(M5_relax)
for i in range(len(SAM_modes)):
    
    weights = np.dot(SAM_modes[i].flatten(), M5_relax.flatten())
    print(weights)


In [26]:
np.dot((M5_relax-NHNNO_super_unlabled_positions).flatten(),eigenvec.flatten())

NameError: name 'eigenvec' is not defined

In [ ]:
NHNNO_super_unlabled_positions

array([[5.00000000e-01, 0.00000000e+00, 2.50000000e-01],
       [0.00000000e+00, 5.00000000e-01, 2.50000000e-01],
       [5.00000000e-01, 0.00000000e+00, 7.50000000e-01],
       [0.00000000e+00, 5.00000000e-01, 7.50000000e-01],
       [0.00000000e+00, 0.00000000e+00, 1.48650133e-01],
       [5.00000000e-01, 5.00000000e-01, 1.48650133e-01],
       [0.00000000e+00, 0.00000000e+00, 6.48650133e-01],
       [5.00000000e-01, 5.00000000e-01, 6.48650133e-01],
       [0.00000000e+00, 0.00000000e+00, 3.51349867e-01],
       [5.00000000e-01, 5.00000000e-01, 3.51349867e-01],
       [0.00000000e+00, 0.00000000e+00, 8.51349867e-01],
       [5.00000000e-01, 5.00000000e-01, 8.51349867e-01],
       [0.00000000e+00, 0.00000000e+00, 2.50000000e-01],
       [5.00000000e-01, 5.00000000e-01, 2.50000000e-01],
       [0.00000000e+00, 0.00000000e+00, 7.50000000e-01],
       [5.00000000e-01, 5.00000000e-01, 7.50000000e-01],
       [2.50000000e-01, 2.50000000e-01, 3.30940731e-01],
       [7.50000000e-01, 7.50000

## Changing the direction


Do dot product of eigenvec with SAMs, half of the SAMs should have ~0 contributions (0A direction) while other half have non-zero (A0 direction). 

In [ ]:
eigenvec=(
'''
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0.002126729627913385 0.01120859300922065 0
0.002126729627706664 0.01120859300918171 0
-0.002126729627706664 -0.01120859300918171 0
-0.002126729627913385 -0.01120859300922065 0
0.002126729627913385 0.01120859300922065 0
0.002126729627706664 0.01120859300918171 0
-0.002126729627706664 -0.01120859300918171 0
-0.002126729627913385 -0.01120859300922065 0
-0.002126729627706664 0.01120859300918171 0
-0.002126729627913385 0.01120859300922065 0
0.002126729627913385 -0.01120859300922065 0
0.002126729627706664 -0.01120859300918171 0
-0.002126729627706664 0.01120859300918171 0
-0.002126729627913385 0.01120859300922065 0
0.002126729627913385 -0.01120859300922065 0
0.002126729627706664 -0.01120859300918171 0
-1.918738860348284e-12 -0.2080674222547224 0
1.918738860348284e-12 -0.2080674222547224 0
-1.918738860348284e-12 0.2080674222547224 0
1.918738860348284e-12 0.2080674222547224 0
-1.918738860348284e-12 -0.2080674222547224 0
1.918738860348284e-12 -0.2080674222547224 0
-1.918738860348284e-12 0.2080674222547224 0
1.918738860348284e-12 0.2080674222547224 0
-0.001260685419313083 0.04851132014447992 -1.590207089812902e-12
-0.001260685419313083 0.04851132014447992 1.590207089812902e-12
0.001260685419313083 -0.04851132014447992 -1.590207089812902e-12
0.001260685419313083 -0.04851132014447992 1.590207089812902e-12
-0.001260685419313083 0.04851132014447992 -1.590207089812902e-12
-0.001260685419313083 0.04851132014447992 1.590207089812902e-12
0.001260685419313083 -0.04851132014447992 -1.590207089812902e-12
0.001260685419313083 -0.04851132014447992 1.590207089812902e-12
4.474669816631088e-13 -1.156519314952823e-14 -0.1724558940854992
-4.474669816631088e-13 1.156519314952823e-14 -0.1724558940854992
4.474669816631088e-13 -1.156519314952823e-14 0.1724558940854992
-4.474669816631088e-13 1.156519314952823e-14 0.1724558940854992
4.474669816631088e-13 -1.156519314952823e-14 -0.1724558940854992
-4.474669816631088e-13 1.156519314952823e-14 -0.1724558940854992
4.474669816631088e-13 -1.156519314952823e-14 0.1724558940854992
-4.474669816631088e-13 1.156519314952823e-14 0.1724558940854992
4.474669816631088e-13 1.156519314952823e-14 0.1724558940854992
-4.474669816631088e-13 -1.156519314952823e-14 0.1724558940854992
4.474669816631088e-13 1.156519314952823e-14 -0.1724558940854992
-4.474669816631088e-13 -1.156519314952823e-14 -0.1724558940854992
4.474669816631088e-13 1.156519314952823e-14 0.1724558940854992
-4.474669816631088e-13 -1.156519314952823e-14 0.1724558940854992
4.474669816631088e-13 1.156519314952823e-14 -0.1724558940854992
-4.474669816631088e-13 -1.156519314952823e-14 -0.1724558940854992
0.001260685419313083 0.04851132014447992 1.590207089812902e-12
0.001260685419313083 0.04851132014447992 -1.590207089812902e-12
-0.001260685419313083 -0.04851132014447992 1.590207089812902e-12
-0.001260685419313083 -0.04851132014447992 -1.590207089812902e-12
0.001260685419313083 0.04851132014447992 1.590207089812902e-12
0.001260685419313083 0.04851132014447992 -1.590207089812902e-12
-0.001260685419313083 -0.04851132014447992 1.590207089812902e-12
-0.001260685419313083 -0.04851132014447992 -1.590207089812902e-12
0.01165506108697134 0.09185665203534664 0
0.01165506108527744 0.09185665203513206 0
-0.01165506108527744 -0.09185665203513206 0
-0.01165506108697134 -0.09185665203534664 0
0.01165506108697134 0.09185665203534664 0
0.01165506108527744 0.09185665203513206 0
-0.01165506108527744 -0.09185665203513206 0
-0.01165506108697134 -0.09185665203534664 0
-0.01165506108527744 0.09185665203513206 0
-0.01165506108697134 0.09185665203534664 0
0.01165506108697134 -0.09185665203534664 0
0.01165506108527744 -0.09185665203513206 0
-0.01165506108527744 0.09185665203513206 0
-0.01165506108697134 0.09185665203534664 0
0.01165506108697134 -0.09185665203534664 0
0.01165506108527744 -0.09185665203513206 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0.00423691932089645 -0.003037719580038923 3.181244601420966e-05
0.004236919320952524 -0.003037719580117104 3.181244601362688e-05
-0.004236919320952524 0.003037719580117104 -3.181244601362688e-05
-0.00423691932089645 0.003037719580038923 -3.181244601420966e-05
0.00423691932089645 -0.003037719580038923 3.181244601420966e-05
0.004236919320952524 -0.003037719580117104 3.181244601362688e-05
-0.004236919320952524 0.003037719580117104 -3.181244601362688e-05
-0.00423691932089645 0.003037719580038923 -3.181244601420966e-05
-0.00423691932089645 0.003037719580038923 3.181244601420966e-05
-0.004236919320952524 0.003037719580117104 3.181244601362688e-05
0.004236919320952524 -0.003037719580117104 -3.181244601362688e-05
0.00423691932089645 -0.003037719580038923 -3.181244601420966e-05
-0.00423691932089645 0.003037719580038923 3.181244601420966e-05
-0.004236919320952524 0.003037719580117104 3.181244601362688e-05
0.004236919320952524 -0.003037719580117104 -3.181244601362688e-05
0.00423691932089645 -0.003037719580038923 -3.181244601420966e-05
-0.00423691932089645 -0.003037719580038923 -3.181244601420966e-05
-0.004236919320952524 -0.003037719580117104 -3.181244601362688e-05
0.004236919320952524 0.003037719580117104 3.181244601362688e-05
0.00423691932089645 0.003037719580038923 3.181244601420966e-05
-0.00423691932089645 -0.003037719580038923 -3.181244601420966e-05
-0.004236919320952524 -0.003037719580117104 -3.181244601362688e-05
0.004236919320952524 0.003037719580117104 3.181244601362688e-05
0.00423691932089645 0.003037719580038923 3.181244601420966e-05
0.00423691932089645 0.003037719580038923 -3.181244601420966e-05
0.004236919320952524 0.003037719580117104 -3.181244601362688e-05
-0.004236919320952524 -0.003037719580117104 3.181244601362688e-05
-0.00423691932089645 -0.003037719580038923 3.181244601420966e-05
0.00423691932089645 0.003037719580038923 -3.181244601420966e-05
0.004236919320952524 0.003037719580117104 -3.181244601362688e-05
-0.004236919320952524 -0.003037719580117104 3.181244601362688e-05
-0.00423691932089645 -0.003037719580038923 3.181244601420966e-05
''')
eigenvec = string_to_matrix(eigenvec)

In [ ]:
A0mode = []
OAmode = []
for i in range(len(SAM_modes)):
    
    weights = np.dot(SAM_modes[i].flatten(), eigenvec.flatten())
    #print(weights)
    if weights > 0.0001 or weights<-0.0001:
        #print(i)
        A0mode.append(np.dot(weights, SAM_modes[i]))
        OAmode.append(np.dot(weights, SAM_modes[i-1]))

#print(A0mode)
A0mode = np.sum(np.array(A0mode), axis=0)
add_mode_in_cart = A0mode*0.1 + np.dot(NHNNO_RX_unlabled_positions, NHNNO_RX_lattice)
print(matrix_to_string(np.dot(add_mode_in_cart, np.linalg.inv(NHNNO_RX_lattice))))

OAmode = np.sum(np.array(OAmode), axis=0)
add_mode_in_cart = (OAmode+A0mode)*0.1 + np.dot(NHNNO_RX_unlabled_positions, NHNNO_RX_lattice)
print(matrix_to_string(np.dot(add_mode_in_cart, np.linalg.inv(NHNNO_RX_lattice))))


0.2500000000000000 0.2500000000000000 0.2500000000000000
0.7500000000000000 0.2500000000000000 0.2500000000000000
0.2500000000000000 0.7500000000000000 0.2500000000000000
0.7500000000000000 0.7500000000000000 0.2500000000000000
0.2500000000000000 0.2500000000000000 0.7500000000000000
0.7500000000000000 0.2500000000000000 0.7500000000000000
0.2500000000000000 0.7500000000000000 0.7500000000000000
0.7500000000000000 0.7500000000000000 0.7500000000000000
0.0002126729627810 0.0011208593009201 0.1486516294796871
0.5002126729627810 0.0011208593009201 0.1486516294796871
-0.0002126729627810 0.4988791406990799 0.1486516294796871
0.4997873270372190 0.4988791406990799 0.1486516294796871
0.0002126729627810 0.0011208593009201 0.6486516294796871
0.5002126729627810 0.0011208593009201 0.6486516294796871
-0.0002126729627810 0.4988791406990799 0.6486516294796871
0.4997873270372190 0.4988791406990799 0.6486516294796871
-0.0002126729627810 0.0011208593009201 0.3513483705203129
0.4997873270372190 0.0011208

M5 mode vectorial difference to parent

In [ ]:
NHNNO_RX_unlabled_positions

array([[0.25      , 0.25      , 0.25      ],
       [0.75      , 0.25      , 0.25      ],
       [0.25      , 0.75      , 0.25      ],
       [0.75      , 0.75      , 0.25      ],
       [0.25      , 0.25      , 0.75      ],
       [0.75      , 0.25      , 0.75      ],
       [0.25      , 0.75      , 0.75      ],
       [0.75      , 0.75      , 0.75      ],
       [0.        , 0.        , 0.14865163],
       [0.5       , 0.        , 0.14865163],
       [0.        , 0.5       , 0.14865163],
       [0.5       , 0.5       , 0.14865163],
       [0.        , 0.        , 0.64865163],
       [0.5       , 0.        , 0.64865163],
       [0.        , 0.5       , 0.64865163],
       [0.5       , 0.5       , 0.64865163],
       [0.        , 0.        , 0.35134837],
       [0.5       , 0.        , 0.35134837],
       [0.        , 0.5       , 0.35134837],
       [0.5       , 0.5       , 0.35134837],
       [0.        , 0.        , 0.85134837],
       [0.5       , 0.        , 0.85134837],
       [0.

In [ ]:
print(matrix_to_string(NHNNO_RX_lattice))

7.6783030983169080 0.0000000000000000 0.0000000000000000
0.0000000000000000 7.6783030983169080 0.0000000000000000
0.0000000000000000 0.0000000000000000 22.1545388808033330


In [ ]:
M5_2x2x2 = '''  
0.7500001456499987 0.7615146175351555 0.2500000000000000
0.2500001456499987 0.2615146175351555 0.2500000000000000
0.7499998543500013 0.2384853824648445 0.2500000000000000
0.2499998543500013 0.7384853824648445 0.2500000000000000
0.7500001456499987 0.7615146175351555 0.7500000000000000
0.2500001456499987 0.2615146175351555 0.7500000000000000
0.7499998543500013 0.2384853824648445 0.7500000000000000
0.2499998543500013 0.7384853824648445 0.7500000000000000
0.0032552209661460 0.0013130626763668 0.1496758583448283
0.5032552209661461 0.5013130626763669 0.1496758583448283
0.4967447790338539 0.9986869373236331 0.1496758583448283
0.9967447790338539 0.4986869373236331 0.1496758583448283
0.0032552209661460 0.0013130626763668 0.6496758583448283
0.5032552209661461 0.5013130626763669 0.6496758583448283
0.4967447790338539 0.9986869373236331 0.6496758583448283
0.9967447790338539 0.4986869373236331 0.6496758583448283
0.0032552112161462 0.9986870179736336 0.3503241416551717
0.5032552112161461 0.4986870179736336 0.3503241416551717
0.4967447887838539 0.0013129820263665 0.3503241416551717
0.9967447887838539 0.5013129820263664 0.3503241416551717
0.0032552112161462 0.9986870179736336 0.8503241416551717
0.5032552112161461 0.4986870179736336 0.8503241416551717
0.4967447887838539 0.0013129820263665 0.8503241416551717
0.9967447887838539 0.5013129820263664 0.8503241416551717
0.9425061768073455 0.9999999897999992 0.2500000000000000
0.4425061768073455 0.4999999897999992 0.2500000000000000
0.5574938231926545 0.0000000102000008 0.2500000000000000
0.0574938231926545 0.5000000102000008 0.2500000000000000
0.9425061768073455 0.9999999897999992 0.7500000000000000
0.4425061768073455 0.4999999897999992 0.7500000000000000
0.5574938231926545 0.0000000102000008 0.7500000000000000
0.0574938231926545 0.5000000102000008 0.7500000000000000
0.2500000000000000 0.0000000000000000 0.3173562518826643
0.7500000000000000 0.5000000000000000 0.3173562518826643
0.7500000000000000 0.0000000000000000 0.3519303673757861
0.2500000000000000 0.5000000000000000 0.3519303673757861
0.2500000000000000 0.0000000000000000 0.8173562518826643
0.7500000000000000 0.5000000000000000 0.8173562518826643
0.7500000000000000 0.0000000000000000 0.8519303673757861
0.2500000000000000 0.5000000000000000 0.8519303673757861
0.5000000000000000 0.7500000000000000 0.3308788866235233
0.0000000000000000 0.2500000000000000 0.3308788866235233
0.5000000000000000 0.2500000000000000 0.3274543786795334
0.0000000000000000 0.7500000000000000 0.3274543786795334
0.5000000000000000 0.7500000000000000 0.8308788866235233
0.0000000000000000 0.2500000000000000 0.8308788866235233
0.5000000000000000 0.2500000000000000 0.8274543786795334
0.0000000000000000 0.7500000000000000 0.8274543786795334
0.5000000000000000 0.7500000000000000 0.1725456950204602
0.0000000000000000 0.2500000000000000 0.1725456950204602
0.5000000000000000 0.2500000000000000 0.1691210096764735
0.0000000000000000 0.7500000000000000 0.1691210096764735
0.5000000000000000 0.7500000000000000 0.6725456950204602
0.0000000000000000 0.2500000000000000 0.6725456950204602
0.5000000000000000 0.2500000000000000 0.6691210096764735
0.0000000000000000 0.7500000000000000 0.6691210096764735
0.2500000000000000 0.0000000000000000 0.1826435810173308
0.7500000000000000 0.5000000000000000 0.1826435810173308
0.7500000000000000 0.0000000000000000 0.1480697697242093
0.2500000000000000 0.5000000000000000 0.1480697697242093
0.2500000000000000 0.0000000000000000 0.6826435810173308
0.7500000000000000 0.5000000000000000 0.6826435810173308
0.7500000000000000 0.0000000000000000 0.6480697697242093
0.2500000000000000 0.5000000000000000 0.6480697697242093
0.0471512776911212 0.0144468733485895 0.0704324538687757
0.5471512776911212 0.5144468733485894 0.0704324538687757
0.4528487223088788 0.9855531266514106 0.0704324538687757
0.9528487223088788 0.4855531266514105 0.0704324538687757
0.0471512776911212 0.0144468733485895 0.5704324538687757
0.5471512776911212 0.5144468733485894 0.5704324538687757
0.4528487223088788 0.9855531266514106 0.5704324538687757
0.9528487223088788 0.4855531266514105 0.5704324538687757
0.0471515821411223 0.9855526827014067 0.4295675461312243
0.5471515821411224 0.4855526827014067 0.4295675461312243
0.4528484178588776 0.0144473172985932 0.4295675461312243
0.9528484178588776 0.5144473172985933 0.4295675461312243
0.0471515821411223 0.9855526827014067 0.9295675461312243
0.5471515821411224 0.4855526827014067 0.9295675461312243
0.4528484178588776 0.0144473172985932 0.9295675461312243
0.9528484178588776 0.5144473172985933 0.9295675461312243
0.2500003522499981 0.2295032189847462 0.0000000000000000
0.7500003522499981 0.7295032189847461 0.0000000000000000
0.2499996477500020 0.7704967810152539 0.0000000000000000
0.7499996477500019 0.2704967810152539 0.0000000000000000
0.2500003522499981 0.2295032189847462 0.5000000000000000
0.7500003522499981 0.7295032189847461 0.5000000000000000
0.2499996477500020 0.7704967810152539 0.5000000000000000
0.7499996477500019 0.2704967810152539 0.5000000000000000
0.3280947984251910 0.3071843997799348 0.0266803689215109
0.8280947984251910 0.8071843997799348 0.0266803689215109
0.3296467831673446 0.8517489461062873 0.0276619378462755
0.8296467831673446 0.3517489461062873 0.0276619378462755
0.3280947984251910 0.3071843997799348 0.5266803689215038
0.8280947984251910 0.8071843997799348 0.5266803689215038
0.3296467831673446 0.8517489461062873 0.5276619378462755
0.8296467831673446 0.3517489461062873 0.5276619378462755
0.1703530978326562 0.1482510688937033 0.0276619378462755
0.6703530978326562 0.6482510688937033 0.0276619378462755
0.6719050825748099 0.1928156152200557 0.0266803689215109
0.1719050825748099 0.6928156152200557 0.0266803689215109
0.1703530978326562 0.1482510688937033 0.5276619378462755
0.6703530978326562 0.6482510688937033 0.5276619378462755
0.6719050825748099 0.1928156152200557 0.5266803689215038
0.1719050825748099 0.6928156152200557 0.5266803689215038
0.6719057731747995 0.8071843951799309 0.4733195490784752
0.1719057731747995 0.3071843951799309 0.4733195490784752
0.1703526752326426 0.8517491887062931 0.4723381411537204
0.6703526752326425 0.3517491887062931 0.4723381411537204
0.6719057731747995 0.8071843951799309 0.9733195490784894
0.1719057731747995 0.3071843951799309 0.9733195490784894
0.1703526752326426 0.8517491887062931 0.9723381411537204
0.6703526752326425 0.3517491887062931 0.9723381411537204
0.3296473397673445 0.1482508112937104 0.4723379026537273
0.8296473397673445 0.6482508112937104 0.4723379026537273
0.3280942418251911 0.6928156048200691 0.4733193105784821
0.8280942418251911 0.1928156048200691 0.4733193105784821
0.3296473397673445 0.1482508112937104 0.9723379026537273
0.8296473397673445 0.6482508112937104 0.9723379026537273
0.3280942418251911 0.6928156048200691 0.9733193105784963
0.8280942418251911 0.1928156048200691 0.9733193105784963
'''

M5_2x2x2direct = string_to_matrix(M5_2x2x2)
M5_2x2x2_lattice = '''  
7.7275319783887157 -0.0207147730314252 0.0000000000000000
-0.0207147730314252 7.7275319783887157 0.0000000000000000
0.0000000000000000 0.0000000000000000 22.0445890945727747
'''
M5_2x2x2_lattice = string_to_matrix(M5_2x2x2_lattice)
M5_2x2x2cart = np.dot(M5_2x2x2direct, M5_2x2x2_lattice)
NHNNO_RX_cart = np.dot(NHNNO_RX_unlabled_positions, NHNNO_RX_lattice)
diff = (M5_2x2x2cart-NHNNO_RX_cart)
print(diff)
diff_vect = np.dot((M5_2x2x2cart-NHNNO_RX_cart),np.linalg.inv(NHNNO_RX_lattice))
diff_vect

[[ 3.86025922e+00  3.94947619e+00 -2.76939124e-02]
 [-3.83238197e+00  9.60675825e-02 -2.76939124e-02]
 [ 3.87109140e+00 -3.93148153e+00 -2.76939124e-02]
 [-3.84226456e+00 -5.73581547e-02 -2.76939124e-02]
 [ 3.86025922e+00  3.94947619e+00 -8.30817371e-02]
 [-3.83238197e+00  9.60675825e-02 -8.30817371e-02]
 [ 3.87109140e+00 -3.93148153e+00 -8.30817371e-02]
 [-3.84226456e+00 -5.73581547e-02 -8.30817371e-02]
 [ 2.51276243e-02  1.00793027e-02  6.11172367e-03]
 [ 3.93036450e-02  3.86348791e+00  6.11172367e-03]
 [ 3.81792359e+00  3.86786271e+00  6.11172367e-03]
 [ 3.85281439e+00 -6.26066853e-03  6.11172367e-03]
 [ 2.51276243e-02  1.00793027e-02 -4.92761011e-02]
 [ 3.93036450e-02  3.86348791e+00 -4.92761011e-02]
 [ 3.81792359e+00  3.86786271e+00 -4.92761011e-02]
 [ 3.85281439e+00 -6.26066853e-03 -4.92761011e-02]
 [ 4.46717386e-03  7.71731844e+00 -6.14995484e-02]
 [ 3.93579675e-02  3.84319506e+00 -6.14995484e-02]
 [ 3.83858404e+00 -3.83937643e+00 -6.14995484e-02]
 [ 3.85276006e+00  1.40321757e-

array([[ 5.02738390e-01,  5.14357505e-01, -1.24998675e-03],
       [-4.99107815e-01,  1.25113002e-02, -1.24998675e-03],
       [ 5.04149112e-01, -5.12013983e-01, -1.24998675e-03],
       [-5.00394868e-01, -7.47000260e-03, -1.24998675e-03],
       [ 5.02738390e-01,  5.14357505e-01, -3.74996025e-03],
       [-4.99107815e-01,  1.25113002e-02, -3.74996025e-03],
       [ 5.04149112e-01, -5.12013983e-01, -3.74996025e-03],
       [-5.00394868e-01, -7.47000260e-03, -3.74996025e-03],
       [ 3.27248008e-03,  1.31267154e-03,  2.75857506e-04],
       [ 5.11868507e-03,  5.03158877e-01,  2.75857506e-04],
       [ 4.97224837e-01,  5.03728626e-01,  2.75857506e-04],
       [ 5.01768817e-01, -8.15354162e-04,  2.75857506e-04],
       [ 3.27248008e-03,  1.31267154e-03, -2.22411599e-03],
       [ 5.11868507e-03,  5.03158877e-01, -2.22411599e-03],
       [ 4.97224837e-01,  5.03728626e-01, -2.22411599e-03],
       [ 5.01768817e-01, -8.15354162e-04, -2.22411599e-03],
       [ 5.81779531e-04,  1.00506003e+00

## FROZSL modes

In [ ]:

# Splitting sections based on double newline characters
sections = GM5minus_primitive.strip().split('\n\n')

# Initialize an empty list to store arrays
arrays = []

# Iterate over sections
for section in sections:
    # Split each section into rows
    rows = section.split('\n')
    # Initialize an empty list to store rows
    section_array = []
    # Iterate over rows
    for row in rows:
        # Split each row into elements and convert them to floats
        elements = [float(elem) for elem in row.split()]
        # Append the row to the section array
        section_array.append(elements)
    # Convert the section array to a numpy array and append to the list of arrays
    arrays.append(np.array(section_array))

# Convert the list of arrays to a 3D numpy array
GM5minus_primitive = np.array(arrays)

# Print the resulting 3D array
print(GM5minus_primitive)

NameError: name 'GM5minus_primitive' is not defined

In [ ]:
def align_positions(base_array, target_array, additional_array, tolerance=1e-4):
    """
    Reorder the rows of target_array and additional_array to match the order in base_array, 
    considering numerical differences within a specified tolerance.

    :param base_array: The base numpy array with the desired row order.
    :param target_array: The first target numpy array to be reordered.
    :param additional_array: The second target numpy array to be reordered in the same way as target_array.
    :param tolerance: The tolerance level for numerical comparison.
    :return: Reordered versions of target_array and additional_array.
    """
    indices = []
    for row in base_array:
        match = np.where(np.all(np.isclose(target_array, row, atol=tolerance), axis=1))[0]
        if match.size == 0:
            raise ValueError(f"No matching row found in target_array for row: {row}")
        indices.append(match[0])

    # Reordering target_array and additional_array based on these indices
    aligned_target_array = target_array[indices]
    aligned_additional_array = additional_array[indices]

    # Debug check: Ensure that the rows are close enough
    if not np.all(np.isclose(base_array, aligned_target_array, atol=tolerance)):
        raise ValueError("The target array could not be aligned closely enough with the base array.")

    return aligned_target_array, aligned_additional_array

In [ ]:
data_str = """
0.0000000000000000  0.0000000000000000  0.5000000000000000
0.0000000000000000  0.0000000000000000  0.0000000000000000
0.5000000000000000  0.5000000000000000  0.2005539978064235
0.5000000000000000  0.5000000000000000  0.7994459721935741
0.0000000000000000  0.5000000000000000  0.1608807830483698
0.0000000000000000  0.5000000000000000  0.8391192169516302
0.5000000000000000  0.0000000000000000  0.1608807830483698
0.5000000000000000  0.0000000000000000  0.8391192169516302
0.5000000000000000  0.5000000000000000  0.3615703837746054
0.5000000000000000  0.5000000000000000  0.6384296462253971
0.5000000000000000  0.5000000000000000  0.0000000000000000
"""
# Splitting the data into rows and then into elements, converting each to a float
P4mmm_prim = [list(map(float, row.split())) for row in data_str.strip().split('\n')]

# Converting the data to a NumPy matrix
P4mmm_prim = np.array(P4mmm_prim)


frozsl_pos = """
    0.000000000000000   0.000000000000000   0.500000000000000
    0.000000000000000   0.000000000000000   0.000000000000000
    0.500000000000000   0.500000000000000   0.200580000000000
    0.500000000000000   0.500000000000000   0.799420000000000
    0.000000000000000   0.500000000000000   0.160890000000000
    0.500000000000000   0.000000000000000   0.839110000000000
    0.500000000000000   0.000000000000000   0.160890000000000
   -0.000000000000000   0.500000000000000   0.839110000000000
    0.500000000000000   0.500000000000000   0.361610000000000
    0.500000000000000   0.500000000000000   0.638390000000000
    0.500000000000000   0.500000000000000   0.000000000000000
"""
# Splitting the data into rows and then into elements, converting each to a float
frozsl_pos = [list(map(float, row.split())) for row in frozsl_pos.strip().split('\n')]

# Converting the data to a NumPy matrix
frozsl_pos = np.array(frozsl_pos)


In [ ]:
# Input data as a multiline string, including labels
primitive_pos = """
Rb  0.0000000000000000  0.0000000000000000  0.5000000000000000
Nd  0.0000000000000000  0.0000000000000000  0.0000000000000000
Nb  0.5000000000000000  0.5000000000000000  0.2005539978064235
Nb  0.5000000000000000  0.5000000000000000  0.7994459721935741
O  0.0000000000000000  0.5000000000000000  0.1608807830483698
O  0.0000000000000000  0.5000000000000000  0.8391192169516302
O  0.5000000000000000  0.0000000000000000  0.1608807830483698
O  0.5000000000000000  0.0000000000000000  0.8391192169516302
O  0.5000000000000000  0.5000000000000000  0.3615703837746054
O  0.5000000000000000  0.5000000000000000  0.6384296462253971
O  0.5000000000000000  0.5000000000000000  0.000000000000000
"""

def matrix_to_string(matrix):
        # Update this function to handle labels correctly
        row_strings = [" ".join(str(value) for value in row) for row in matrix]

        return "\n".join(row_strings)

def add_mode(primitive, mode):

        labels, primitive_data = [], []

        for row in primitive.strip().split('\n'):
                elements = row.split()
                labels.append(elements[0])  # Extract label
                primitive_data.append(list(map(float, elements[1:])))  # Convert remaining elements to floats

        cart_mode = np.dot(mode,lattice_basis)

        normalized_mode = cart_mode / np.linalg.norm(cart_mode) # normalize in cartesian space

        #print(matrix_to_string(normalized_mode),'\n')

        direct = np.dot(normalized_mode, np.linalg.inv(lattice_basis))
        #print('Mode in direct after notmalization:')
        #print(direct)
        distorted_cart = primitive_data + 0.01 * direct
        #print(distorted_cart)
        return distorted_cart

#print(P4mmm_prim, frozsl_pos)


for i in range(len(GM5minus_primitive)):
    positions, mode = align_positions(P4mmm_prim, frozsl_pos, GM5minus_primitive[i])
    #print(matrix_to_string(mode / np.linalg.norm(mode)), '\n')
    moded = add_mode(primitive_pos, mode)
    formatted_num = matrix_to_string(moded)
    print(formatted_num,'\n')

## Checking Orthonormality

In [ ]:


import numpy as np
def split_modes(input_data, columns_to_remove=None):
    data_arrays = []

    # Split the input data into individual arrays
    arrays = input_data.strip().split('\n\n')

    for array in arrays:
        lines = array.split('\n')

        # Remove the first line
        header = lines.pop(0)

        data = []
        for line in lines:
            if line.strip():  # Check if the line is not empty
                components = line.split()
                # Skip the first component and convert the rest to floats
                row = [components[0]] + [float(component) for component in components[1:]]
                
                data.append(row)

        # Remove specific columns if specified
        if columns_to_remove:
            for row in data:
                for col_index in sorted(columns_to_remove, reverse=True):
                    # Adjusted to account for the skipped first column
                    if col_index != 0:
                        del row[col_index]

        # Construct the data array dictionary
        data_arrays.append({'header': header, 'data': data})

    return data_arrays

def add_distortions_by_labels(matrix1, matrix2, matrix3, lattice_basis):
    """
    Adds distortions to rows in matrix1 based on corresponding rows in matrix2.

    Args:
    - matrix1: First array with rows to be distorted
    - matrix2: Second array with distortions and labels
    - lattice_basis: Basis matrix for transformations

    Returns:
    - result_matrix: Matrix1 with distortions added
    """

    # Convert matrix2 to Cartesian coordinates
    cartesian_matrix2 = np.dot(matrix2[:, 1:].astype(float), lattice_basis)

    # Convert matrix3 to Cartesian coordinates
    cartesian_matrix3 = np.dot(matrix3[:, 1:].astype(float), lattice_basis)

    if len(cartesian_matrix2) == len(cartesian_matrix3):
        cartesian_matrix_pos = cartesian_matrix2 + cartesian_matrix3
        # Normalize entire matrix2
        cartesian_matrix_pos = cartesian_matrix_pos / np.linalg.norm(cartesian_matrix_pos)#, axis=1)[:, np.newaxis]

        cartesian_matrix_neg = cartesian_matrix2 - cartesian_matrix3
        # Normalize entire matrix2
        cartesian_matrix_neg = cartesian_matrix_neg / np.linalg.norm(cartesian_matrix_neg)#, axis=1)[:, np.newaxis]

    else:
        return 0

    shape_matrix1 = matrix1[:, 1:].astype(float).shape
    # Create a new empty matrix of the same shape as matrix1
    empty_matrix_pos = np.zeros(shape_matrix1)

    matrix_copy = np.copy(matrix1[:, 1:].astype(float))
    for row, normalized_row in zip(matrix2, cartesian_matrix_pos):
        label = row[0]
        for i, row1 in enumerate(matrix1):
            if row1[0] == label:
                #print('FOund a match!')
                inverse_t = np.linalg.inv(lattice_basis)
                direct = np.dot(normalized_row, lattice_basis)
                
                empty_matrix_pos[i] += normalized_row
                #distortion = direct * 0.001
                #matrix_copy[i] += distortion
    result_pos = np.hstack((matrix1[:, :1], empty_matrix_pos))

    shape_matrix1 = matrix1[:, 1:].astype(float).shape
    # Create a new empty matrix of the same shape as matrix1
    empty_matrix_neg = np.zeros(shape_matrix1)

    matrix_copy = np.copy(matrix1[:, 1:].astype(float))
    for row, normalized_row in zip(matrix2, cartesian_matrix_neg):
        label = row[0]
        for i, row1 in enumerate(matrix1):
            if row1[0] == label:

                inverse_t = np.linalg.inv(lattice_basis)
                direct = np.dot(normalized_row, lattice_basis)

                empty_matrix_neg[i] += normalized_row
                #matrix_copy[i] += distortion
    result_neg = np.hstack((matrix1[:, :1], empty_matrix_neg))

    return empty_matrix_pos, empty_matrix_neg

def add_numbers2labels(data):
    data = np.array(data)

    # Dictionary to keep track of label counts
    label_counts = {}

    # Iterate through the array and append numbers to labels
    for i, row in enumerate(data):
        label = row[0]
        if label not in label_counts:
            label_counts[label] = 1
        else:
            label_counts[label] += 1
        data[i][0] = f"{label}{label_counts[label]}"

    return data

def matrix_to_string(matrix):
    # Update this function to handle labels correctly
    row_strings = [" ".join(str(value) for value in row) for row in matrix]
    return "\n".join(row_strings)

# Process the data with specified columns to remove
result = split_modes(GM5m_mode, [1,2,3])

np.set_printoptions( precision=8, threshold=np.inf,suppress=True,linewidth=1000)

for i in range(0, len(result), 2):

    moded_matrix_pos, moded_matrix_neg = add_distortions_by_labels(np.array([row[:] for row in add_numbers2labels(labeled_matrix)]), np.array(result[i]['data']), np.array(result[i+1]['data']), lattice_basis) #labled matrix had columns of strings - convert to float    
    #    moded_matrix = add_distortions_by_labels(moded_matrix, np.array(result[i+1]['data']),lattice_basis)

    labels = np.array(labeled_matrix[:, 0]).reshape(-1, 1) 
    if not np.all(moded_matrix_pos == 0):  # Check if all elements are not zeros

        formatted_unlabeled = matrix_to_string(moded_matrix_pos)
        #print(formatted_unlabeled,'\n')

        formatted_unlabeled = matrix_to_string(moded_matrix_neg)
        #print(formatted_unlabeled,'\n')
        #print(np.dot(moded_matrix_neg.flatten(),moded_matrix_neg.flatten()))

# List to store all matrices
all_matrices = []

# Loop to calculate and save all matrices
for i in range(0, len(result), 2):
    moded_matrix_pos, moded_matrix_neg = add_distortions_by_labels(
        np.array([row[:] for row in add_numbers2labels(labeled_matrix)]),
        np.array(result[i]['data']), np.array(result[i+1]['data']), lattice_basis
    )
    if not np.all(moded_matrix_pos == 0):  # Check if all elements are not zeros
        all_matrices.append((moded_matrix_pos, moded_matrix_neg))

# Double loop to iterate over saved matrices and perform dot product
for i in range(len(all_matrices)):
    for j in range(i, len(all_matrices)):  # Start from i+1 to avoid redundant computations
        matrix1_pos, matrix1_neg = all_matrices[i]
        matrix2_pos, matrix2_neg = all_matrices[j]
        
        # Dot product between matrices
        dot_product_pos = np.dot(matrix1_pos.flatten(), matrix2_pos.flatten())
        dot_product_neg = np.dot(matrix1_neg.flatten(), matrix2_neg.flatten())

        print(f"Dot product between matrices {i+1} and {j+1} (positive): {dot_product_pos}")
        print(f"Dot product between matrices {i+1} and {j+1} (negative): {dot_product_neg}")  
        dot_product_neg = np.dot(matrix1_neg.flatten(), matrix2_pos.flatten())
        dot_product_pos = np.dot(matrix1_pos.flatten(), matrix2_neg.flatten())


        print(f"Dot product between matrices {i+1} negative and {j+1} (positive): {dot_product_pos}")
        print(f"Dot product between matrices {i+1} positive and {j+1} (negative): {dot_product_neg}")        


Dot product between matrices 1 and 1 (positive): 1.0
Dot product between matrices 1 and 1 (negative): 1.0
Dot product between matrices 1 negative and 1 (positive): 0.0
Dot product between matrices 1 positive and 1 (negative): 0.0
Dot product between matrices 1 and 2 (positive): 0.0
Dot product between matrices 1 and 2 (negative): 0.0
Dot product between matrices 1 negative and 2 (positive): 0.0
Dot product between matrices 1 positive and 2 (negative): 0.0
Dot product between matrices 1 and 3 (positive): 0.0
Dot product between matrices 1 and 3 (negative): 0.0
Dot product between matrices 1 negative and 3 (positive): 0.0
Dot product between matrices 1 positive and 3 (negative): 0.0
Dot product between matrices 1 and 4 (positive): 0.0
Dot product between matrices 1 and 4 (negative): 0.0
Dot product between matrices 1 negative and 4 (positive): 0.0
Dot product between matrices 1 positive and 4 (negative): 0.0
Dot product between matrices 1 and 5 (positive): 0.0
Dot product between matrice